# 🐍 Python Learning Quest (Beginner → Advanced)

**What to do**: work through exercises in numeric order (1.1 → 7.3).

## Recommended flow
1. Run **Environment Setup** once.
2. For each exercise: **Prompt → Starter → Hint → Solution**.
3. Use the **Gamification Dashboard** to track XP/progress.

---


---
# 🎓 BEGINNER EXERCISES: Introduction to Python

This section introduces fundamental Python concepts with **3-tier progressive exercises**:
- **Basic** (⭐): Core concepts, simple syntax
- **Intermediate** (⭐⭐): Applied problems, multiple concepts
- **Advanced** (⭐⭐⭐): Real-world scenarios, optimization

---

## ✅ Setup (run once)

Run the **next cell** once to install the required libraries.



After it finishes, you can collapse that cell and continue with the exercises.



If `%pip` doesn’t work in your environment, run this in a terminal instead:



```bash

python -m pip install numpy pandas matplotlib scikit-learn seaborn pytest

```


In [ ]:
# 🔧 Environment Setup (run once)



# Run this cell once at the start, then collapse it.

# If %pip fails for you, use the terminal command shown in the cell above.



%pip install numpy pandas matplotlib scikit-learn seaborn pytest



print("✅ Setup complete. You can start the exercises!")


## 🕹️ Gamification Setup (XP, Levels, Achievements)
This notebook includes an RPG-style progression system.

- Earn XP per exercise (Basic/Intermediate/Advanced)
- Get bonuses for speed + passing tests
- Maintain daily streaks
- Unlock achievements
- Progress is saved in your browser (Export/Import panel + optional localStorage) — no filesystem writes
---

In [ ]:
# ======================================
# Gamification Engine (Single-file, no filesystem)
# ======================================

from __future__ import annotations

import datetime as _dt
import json as _json
from dataclasses import dataclass, field


def _today_iso() -> str:
    return _dt.date.today().isoformat()


@dataclass
class PlayerProfile:
    name: str
    xp: int = 0
    level: int = 1
    streak: int = 0
    last_completed_date: str | None = None
    completed_exercises: set[str] = field(default_factory=set)
    achievements: set[str] = field(default_factory=set)


class GamificationEngine:
    """In-notebook (single-file) gamification. No module imports, no disk writes."""

    def __init__(self, player_name: str = "Python_Learner", state: dict | None = None):
        self.player = PlayerProfile(name=player_name)
        if state is not None:
            self.load_state(state)

    @staticmethod
    def xp_to_level(xp: int) -> int:
        return max(1, 1 + xp // 100)

    def award_xp(self, amount: int, reason: str = "") -> None:
        amount = int(max(0, amount))
        self.player.xp += amount
        self.player.level = self.xp_to_level(self.player.xp)
        self._check_achievements()

    def mark_completed(self, exercise_id: str) -> None:
        today = _today_iso()
        if self.player.last_completed_date != today:
            if self.player.last_completed_date is not None:
                last = _dt.date.fromisoformat(self.player.last_completed_date)
                self.player.streak = (self.player.streak + 1) if ((_dt.date.today() - last).days == 1) else 1
            else:
                self.player.streak = 1
            self.player.last_completed_date = today

        self.player.completed_exercises.add(str(exercise_id))
        self._check_achievements()

    def _check_achievements(self) -> None:
        completed = len(self.player.completed_exercises)
        if completed >= 1:
            self.player.achievements.add("First Steps")
        if completed >= 10:
            self.player.achievements.add("On a Roll")
        if completed >= 25:
            self.player.achievements.add("Quest Veteran")
        if self.player.streak >= 3:
            self.player.achievements.add("3-Day Streak")
        if self.player.streak >= 7:
            self.player.achievements.add("7-Day Streak")

    def daily_quest(self) -> dict:
        day = _dt.date.today().toordinal()
        quests = [
            {"title": "Warm-up", "metric": "exercises", "goal": 1, "reward_xp": 10},
            {"title": "Consistency", "metric": "exercises", "goal": 3, "reward_xp": 25},
            {"title": "Deep Work", "metric": "exercises", "goal": 5, "reward_xp": 45},
        ]
        return quests[day % len(quests)]

    def to_dict(self) -> dict:
        p = self.player
        return {
            "player": {
                "name": p.name,
                "xp": p.xp,
                "level": p.level,
                "streak": p.streak,
                "last_completed_date": p.last_completed_date,
                "completed_exercises": sorted(p.completed_exercises),
                "achievements": sorted(p.achievements),
            },
        }

    def load_state(self, state: dict) -> None:
        p = state.get("player", {}) if isinstance(state, dict) else {}
        self.player.name = str(p.get("name", self.player.name))
        self.player.xp = int(p.get("xp", 0) or 0)
        self.player.level = self.xp_to_level(self.player.xp)
        self.player.streak = int(p.get("streak", 0) or 0)
        self.player.last_completed_date = p.get("last_completed_date")
        self.player.completed_exercises = set(map(str, p.get("completed_exercises", []) or []))
        self.player.achievements = set(map(str, p.get("achievements", []) or []))
        self._check_achievements()

    def export_json(self) -> str:
        return _json.dumps(self.to_dict(), indent=2, ensure_ascii=False, sort_keys=True)

    def import_json(self, json_text: str) -> None:
        self.load_state(_json.loads(json_text))

    def render_text_dashboard(self) -> str:
        p = self.player
        return (
            f"Player: {p.name} | Level: {p.level} | XP: {p.xp}\n"
            f"Completed: {len(p.completed_exercises)} | Streak: {p.streak}\n"
            f"Achievements: {', '.join(sorted(p.achievements)) if p.achievements else '—'}"
        )


class ExerciseTracker:
    DIFFICULTY_XP = {"Basic": 20, "Intermediate": 40, "Advanced": 60}

    def __init__(self, engine: GamificationEngine):
        self.engine = engine

    def complete(self, exercise_id: str, difficulty: str = "Basic", passed_tests: bool = True, seconds: float | None = None) -> int:
        base = self.DIFFICULTY_XP.get(str(difficulty), 20)
        bonus = (5 if passed_tests else 0)
        if seconds is not None:
            try:
                s = float(seconds)
                bonus += 10 if s <= 60 else (5 if s <= 180 else 0)
            except Exception:
                pass
        earned = base + bonus
        self.engine.award_xp(earned, reason=f"Exercise {exercise_id}")
        self.engine.mark_completed(exercise_id)
        return earned


game = GamificationEngine(player_name="Python_Learner")
tracker = ExerciseTracker(game)

print("✅ Gamification ready (single-file)!")
print(game.render_text_dashboard())
quest = game.daily_quest()
print("Today's quest:", quest['title'], "| Goal:", quest['goal'], quest['metric'], "| Reward:", quest['reward_xp'], "XP")


def show_progress_json() -> None:
    print(game.export_json())


def load_progress_json(json_text: str) -> None:
    game.import_json(json_text)
    print("✅ Progress loaded")
    print(game.render_text_dashboard())


def render_progress_panel(storage_key: str = "python_learning_quest_state_v1") -> None:
    try:
        from IPython.display import display, HTML
    except Exception:
        print("Use show_progress_json() and load_progress_json(...) instead.")
        return

    html = f"""
<style>
.plq-panel {{ border: 1px solid #2a2a2a; padding: 12px; border-radius: 10px; background: #0b0f17; color: #e8eefc; }}
.plq-panel textarea {{ width: 100%; height: 180px; background: #06090f; color: #d7e3ff; border: 1px solid #223; border-radius: 8px; padding: 8px; font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono', 'Courier New', monospace; }}
.plq-panel button {{ margin: 6px 6px 0 0; padding: 6px 10px; border-radius: 8px; border: 1px solid #2b3a55; background: #101826; color: #e8eefc; cursor: pointer; }}
.plq-panel button:hover {{ background: #15223a; }}
.plq-panel .small {{ opacity: 0.85; font-size: 0.9em; }}
</style>
<div class='plq-panel'>
  <div><b>Progress persistence (single-file)</b></div>
  <div class='small'>Save in browser localStorage, or download/upload JSON. To load into Python: copy textarea into <code>load_progress_json(...)</code>.</div>
  <textarea id='plq_state' placeholder='Paste exported JSON here...'></textarea>
  <div>
    <button onclick="document.getElementById('plq_state').value = ''">Clear</button>
    <button onclick="localStorage.setItem('{storage_key}', document.getElementById('plq_state').value);">Save to browser</button>
    <button onclick="document.getElementById('plq_state').value = localStorage.getItem('{storage_key}') || '';">Load from browser</button>
    <button onclick="(function(){{const t=document.getElementById('plq_state').value||''; const blob=new Blob([t],{{type:'application/json'}}); const a=document.createElement('a'); a.href=URL.createObjectURL(blob); a.download='python_learning_quest_progress.json'; a.click(); URL.revokeObjectURL(a.href);}})();">Download JSON</button>
    <input id='plq_file' type='file' accept='application/json' style='display:none' onchange="(function(e){{const f=e.target.files[0]; if(!f) return; const r=new FileReader(); r.onload=()=>{{document.getElementById('plq_state').value = r.result||'';}}; r.readAsText(f);}})(event)" />
    <button onclick="document.getElementById('plq_file').click();">Upload JSON</button>
  </div>
  <hr style='border:0;border-top:1px solid #223; margin:12px 0'/>
  <div class='small'><b>Tip:</b> Run <code>show_progress_json()</code> to print JSON, then paste it here.</div>
</div>
"""
    display(HTML(html))


render_progress_panel()


---
## 🎨 Beautiful UI/UX Dashboard (Neon Theme)
This section adds a **modern neon UI layer** on top of the gamification engine.

### What you get
- A **HTML dashboard** (cards + progress bar)
- **Exercise cards** (difficulty, XP, status)
- A **dark-themed analytics panel** (matplotlib)

### Usage
1. Run the gamification setup cell above (creates `game` and `tracker`).
2. Run the UI cells below to render the dashboard and charts.
---

In [ ]:
# UI/UX Layer (single-file, no external imports)

from __future__ import annotations

from dataclasses import dataclass
import json as _json


def _safe_display_html(html: str) -> None:
    try:
        from IPython.display import display, HTML
        display(HTML(html))
    except Exception:
        print(html)


def _try_update_global_hud(payload: dict) -> None:
    """Best-effort global HUD.

    In VS Code notebooks, HTML outputs can be sandboxed per-cell, which can prevent
    `position: fixed` from staying visible while scrolling. This tries to attach a HUD
    directly to `document.body` via JavaScript so it stays visible across the notebook.
    """
    try:
        from IPython.display import display, Javascript
    except Exception:
        return

    payload_json = _json.dumps(payload, ensure_ascii=False)
    js = """
(function(){
  const data = %s;
  const ID = 'plq-global-hud-v1';
  let hud = document.getElementById(ID);
  if (!hud) {
    hud = document.createElement('div');
    hud.id = ID;
    document.body.appendChild(hud);
  }
  hud.innerHTML = `
    <style>
      #${ID} {
        position: fixed;
        top: 12px;
        right: 12px;
        width: 320px;
        z-index: 2147483647;
        border: 1px solid #24314a;
        background: linear-gradient(135deg, #0b0f17, #0c1220);
        padding: 10px 12px 12px 12px;
        border-radius: 14px;
        color: #e8eefc;
        box-shadow: 0 10px 30px rgba(0,0,0,0.45);
        font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;
      }
      #${ID} .row { display: flex; align-items: center; justify-content: space-between; gap: 8px; }
      #${ID} .title { font-weight: 900; font-size: 0.98em; }
      #${ID} .btn {
        border: 1px solid #2b3a55; background: #101826; color: #e8eefc;
        padding: 3px 8px; border-radius: 999px; cursor: pointer; font-size: 0.85em;
      }
      #${ID} .grid { display: grid; grid-template-columns: repeat(4, minmax(0, 1fr)); gap: 8px; margin-top: 8px; }
      #${ID} .kpi {
        padding: 8px; border-radius: 12px; background: rgba(255,255,255,0.03);
        border: 1px solid rgba(90,110,150,0.35);
      }
      #${ID} .v { font-size: 1.1em; font-weight: 900; }
      #${ID} .k { opacity: 0.85; color: #a5b4d6; font-size: 0.82em; }
      #${ID}.minimized .grid { display: none; }
    </style>
    <div class='row'>
      <div class='title'>Python Learning Quest — Dashboard</div>
      <div class='row' style='gap:6px'>
        <button class='btn' onclick="(function(){const el=document.getElementById('${ID}'); el.classList.toggle('minimized');})()">min</button>
        <button class='btn' onclick="(function(){const el=document.getElementById('${ID}'); el.remove();})()">x</button>
      </div>
    </div>
    <div class='grid'>
      <div class='kpi'><div class='v'>${data.level}</div><div class='k'>Level</div></div>
      <div class='kpi'><div class='v'>${data.xp}</div><div class='k'>XP</div></div>
      <div class='kpi'><div class='v'>${data.completed}</div><div class='k'>Done</div></div>
      <div class='kpi'><div class='v'>${data.streak}</div><div class='k'>Streak</div></div>
    </div>`;
})();
""" % payload_json
    display(Javascript(js))


def render_focus_mode_toggle() -> None:
    html = """
<style>
/* Focus mode should NOT hide the dashboard HUD */
.plq-focus .plq-card { display: none !important; }
</style>
<div style='margin: 10px 0'>
  <button onclick=\"document.body.classList.toggle('plq-focus');\"
    style='padding:6px 10px;border-radius:8px;border:1px solid #2b3a55;background:#101826;color:#e8eefc;cursor:pointer'>
    Toggle Focus Mode
  </button>
  <span style='opacity:.8;margin-left:8px'>Focus mode hides exercise cards to reduce distraction (dashboard stays visible).</span>
</div>
"""
    _safe_display_html(html)


class BeautifulDashboard:
    def __init__(self, engine):
        self.engine = engine

    def render(self) -> None:
        p = self.engine.player
        payload = {
            "level": int(getattr(p, "level", 1)),
            "xp": int(getattr(p, "xp", 0)),
            "completed": len(getattr(p, "completed_exercises", []) or []),
            "streak": int(getattr(p, "streak", 0)),
        }
        _try_update_global_hud(payload)

        # Inline fallback (always visible in this cell output)
        html = f"""
<style>
:root {{
  --plq-panel: #0b0f17;
  --plq-text: #e8eefc;
  --plq-muted: #a5b4d6;
}}
.plq-dashboard--inline {{
  margin: 10px 0;
  border: 1px solid #24314a;
  background: linear-gradient(135deg, var(--plq-panel), #0c1220);
  padding: 12px;
  border-radius: 14px;
  color: var(--plq-text);
}}
.plq-grid {{ display: grid; grid-template-columns: repeat(4, minmax(0, 1fr)); gap: 8px; }}
.plq-kpi {{ padding: 8px; border-radius: 12px; background: rgba(255,255,255,0.03); border: 1px solid rgba(90,110,150,0.35); }}
.plq-kpi .v {{ font-size: 1.15em; font-weight: 800; }}
.plq-kpi .k {{ opacity: 0.85; color: var(--plq-muted); font-size: 0.85em; }}
.plq-title {{ font-weight: 900; font-size: 1.0em; margin-bottom: 8px; }}
</style>
<div class='plq-dashboard--inline'>
  <div class='plq-title'>Python Learning Quest — Dashboard (inline)</div>
  <div class='plq-grid'>
    <div class='plq-kpi'><div class='v'>{payload['level']}</div><div class='k'>Level</div></div>
    <div class='plq-kpi'><div class='v'>{payload['xp']}</div><div class='k'>XP</div></div>
    <div class='plq-kpi'><div class='v'>{payload['completed']}</div><div class='k'>Done</div></div>
    <div class='plq-kpi'><div class='v'>{payload['streak']}</div><div class='k'>Streak</div></div>
  </div>
</div>
"""
        _safe_display_html(html)


@dataclass
class ExerciseCard:
    exercise_id: str
    topic: str
    difficulty: str
    description: str
    xp_reward: int
    status: str = "available"

    def render(self) -> None:
        badge = {"Basic": "#5ef3ff", "Intermediate": "#ffd54a", "Advanced": "#b26bff"}.get(self.difficulty, "#5ef3ff")
        html = f"""
<div class='plq-card' style='margin:10px 0; padding:12px; border-radius:14px; border:1px solid #24314a; background:#0b0f17; color:#e8eefc'>
  <div style='display:flex; align-items:center; justify-content:space-between; gap:10px'>
    <div style='font-weight:800'>Exercise {self.exercise_id}: {self.topic}</div>
    <div style='display:flex; gap:8px; align-items:center'>
      <span style='padding:2px 8px; border-radius:999px; border:1px solid {badge}; color:{badge}; font-weight:700'>{self.difficulty}</span>
      <span style='opacity:.85'>{self.xp_reward} XP</span>
    </div>
  </div>
  <div style='opacity:.9; margin-top:6px'>{self.description}</div>
  <div style='opacity:.75; margin-top:8px'>Status: <b>{self.status}</b></div>
</div>
"""
        _safe_display_html(html)


render_focus_mode_toggle()
BeautifulDashboard(game).render()


In [ ]:
# --- Smoke test: gamification + UI ---
print("== Smoke test: gamification + UI ==")

# 1) Gamification updates state
before = game.to_dict()
earned = tracker.complete("smoke-1", difficulty="Basic", passed_tests=True, seconds=45)
after = game.to_dict()

assert after["player"]["xp"] == before["player"]["xp"] + earned
assert "smoke-1" in after["player"]["completed_exercises"]
assert after["player"]["level"] >= 1

print("Awarded XP:", earned)
print(game.render_text_dashboard())

# 2) Export/import round-trip
state_json = game.export_json()
g2 = GamificationEngine(player_name="Tmp")
g2.import_json(state_json)
assert g2.to_dict() == game.to_dict()
print("✅ export/import roundtrip OK")

# 3) UI helpers run without error
ExerciseCard(
    exercise_id="smoke-1",
    topic="Smoke Test",
    difficulty="Basic",
    description="Verifies the card renderer runs.",
    xp_reward=earned,
    status="completed",
).render()
BeautifulDashboard(game).render()

---
# 🎓 Beginner Exercises (1.1 → 7.3)
Each exercise is organized as: **Prompt → Starter → Hint → Solution**.
---


## 📝 Exercise 1.1: Hello World & Comments ⭐ (Basic)

**Learning Objective**: Understand `print()` function and comment syntax.

### Challenge:
Write a script that:
1. Prints `"Hello, World!"`
2. Adds a single-line comment explaining the `print()` function
3. Adds a multi-line comment describing what the script does

**Expected Output**:
```
Hello, World!
```

In [ ]:
# --- Starter (Exercise 1.1) ---

# Write your solution here first.

# TODO: your code
print("Hello, World!")


# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 1.1.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

# The print() function outputs text to the console
print("Hello, World!")

"""
This script demonstrates:
1. Basic print() function usage
2. Single-line comments with #
3. Multi-line comments with triple quotes
"""
```
</details>


## 📝 Exercise 1.2: Code Snippets ⭐⭐ (Intermediate)

**Learning Objective**: Master different string output methods.

### Challenge:
Create 3 separate code blocks demonstrating:
1. Multi-line string printing
2. Variable concatenation using `+`
3. f-string formatting

**Constraint**: No repeated syntax; demonstrate variety.

**Expected Output**:
```
This is a
multi-line
string

Name: Alice, Age: 30
Name: Alice, Age: 30  (using f-string)
```

In [ ]:
# --- Starter (Exercise 1.2) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 1.2.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

# 1. Multi-line string
print("""This is a
multi-line
string""")

# 2. Variable concatenation
name = "Alice"
age = 30
print("Name: " + name + ", Age: " + str(age))

# 3. f-string formatting (modern Python)
print(f"Name: {name}, Age: {age}  (using f-string)")
```
</details>


## 📝 Exercise 1.3: Interactive Personal Summary ⭐⭐⭐ (Advanced)

**Learning Objective**: Handle user input with validation and type information.

### Challenge:
Write a script that:
1. Takes `input()` for: name, age, city
2. Handles empty inputs gracefully (replace with default values)
3. Constructs a formatted bio using f-strings
4. Prints each value with its type using `type()`

**Constraint**: Handle edge cases like empty strings.

**Expected Output**:
```
═══════════════════════════
Personal Summary
═══════════════════════════
Name:  Alice (type: str)
Age:   25 (type: str)
City:  NYC (type: str)
═══════════════════════════
```

In [ ]:
# --- Starter (Exercise 1.3) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 1.3.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

def create_personal_summary():
    """Interactive personal summary with input validation."""
    
    # Get input with default fallbacks
    name = input("Enter your name: ").strip() or "Anonymous"
    age = input("Enter your age: ").strip() or "Unknown"
    city = input("Enter your city: ").strip() or "Unknown"
    
    # Format bio with type information
    bio = f"""
═══════════════════════════
Personal Summary
═══════════════════════════
Name:  {name} (type: {type(name).__name__})
Age:   {age} (type: {type(age).__name__})
City:  {city} (type: {type(city).__name__})
═══════════════════════════
    """
    
    print(bio)

# Uncomment to run interactively:
# create_personal_summary()
```
</details>


## 📝 Exercise 2.1: Variable Types ⭐ (Basic)

**Learning Objective**: Understand Python's dynamic typing system.

### Challenge:
Create 5 variables of different types:
1. Integer
2. Float
3. String
4. Boolean
5. None

Print each with its value and type using `type()`.

**Expected Output**:
```
Value: 42         | Type: <class 'int'>
Value: 3.14159    | Type: <class 'float'>
Value: Python     | Type: <class 'str'>
Value: True       | Type: <class 'bool'>
Value: None       | Type: <class 'NoneType'>
```

In [ ]:
# --- Starter (Exercise 2.1) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 2.1.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

var_int = 42
var_float = 3.14159
var_str = "Python"
var_bool = True
var_none = None

variables = [var_int, var_float, var_str, var_bool, var_none]

for var in variables:
    print(f"Value: {str(var):<10} | Type: {type(var)}")
```
</details>


## 📝 Exercise 2.2: Converting Input Strings ⭐⭐ (Intermediate)

**Goal**: practice type conversion and input validation.

### Task
Implement `convert_inputs(age_text, height_text, subscribed_text)`.

Return a dictionary:
- `age` as an `int`
- `height` as a `float`
- `subscribed` as a `bool`

### Rules
- Strip whitespace from inputs.
- Accept `subscribed_text` values (case-insensitive): `yes/no`, `true/false`, `y/n`, `1/0`.
- Raise `ValueError` with a helpful message for invalid values.

### Example
```python
convert_inputs('21', '1.75', 'YES')
# {'age': 21, 'height': 1.75, 'subscribed': True}
```

In [ ]:
# --- Starter (Exercise 2.2) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here (don’t scroll to the solution cell until you try!).

def convert_inputs(age_text: str, height_text: str, subscribed_text: str) -> dict[str, object]:
    # TODO: implement
    raise NotImplementedError


# Optional quick checks (uncomment after implementing)
# assert convert_inputs("21", "1.75", "yes")["subscribed"] is True
# assert convert_inputs("21", "1.75", "No")["subscribed"] is False
# print("✅ 2.2 OK")


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- `int(age_text)` and `float(height_text)` inside `try/except` to raise a clean `ValueError`.
- For booleans: normalize with `.strip().lower()` then check membership in sets:
  - `truthy = {'true','t','yes','y','1'}`
  - `falsy = {'false','f','no','n','0'}`

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

def convert_inputs(age_text: str, height_text: str, subscribed_text: str) -> dict[str, object]:
    age_text = age_text.strip()
    height_text = height_text.strip()
    subscribed_text = subscribed_text.strip().lower()

    try:
        age = int(age_text)
    except Exception as exc:
        raise ValueError(f"Invalid age_text={age_text!r}. Expected an integer.") from exc

    try:
        height = float(height_text)
    except Exception as exc:
        raise ValueError(f"Invalid height_text={height_text!r}. Expected a number.") from exc

    truthy = {"true", "t", "yes", "y", "1"}
    falsy = {"false", "f", "no", "n", "0"}
    if subscribed_text in truthy:
        subscribed = True
    elif subscribed_text in falsy:
        subscribed = False
    else:
        raise ValueError(
            "Invalid subscribed_text. Use one of: yes/no, true/false, 1/0, y/n (case-insensitive)."
        )

    return {"age": age, "height": height, "subscribed": subscribed}


# Quick checks
assert convert_inputs("21", "1.75", "yes") == {"age": 21, "height": 1.75, "subscribed": True}
assert convert_inputs("21", "1.75", "No") == {"age": 21, "height": 1.75, "subscribed": False}
print("✅ 2.2 OK")
```
</details>


## 📝 Exercise 2.3: Flexible Calculator ⭐⭐⭐ (Advanced)

**Learning Objective**: Dynamic type handling without `eval()`.

### Challenge:
Build a calculator that:
1. Accepts string input: `"10 + 5.5"`, `"20 * True"`
2. Parses & converts types intelligently
3. Handles `TypeError` gracefully
4. Logs type changes throughout

**Constraint**: No `eval()`; use safe parsing only.

**Expected Output**:
```
Input: "10 + 5.5"
Type Log:
  num1: 10 -> int
  num2: 5.5 -> float
  result: 15.5 (float)
Result: 15.5
```

In [ ]:
# --- Starter (Exercise 2.3) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 2.3.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

def flexible_calculator(expression: str):
    """
    Safe calculator without eval().
    
    Args:
        expression (str): Math expression like "10 + 5.5"
    
    Returns:
        tuple: (result, type_log)
    """
    type_log = []
    
    try:
        # Parse safely
        parts = expression.split()
        if len(parts) != 3:
            raise ValueError("Invalid format. Use: 'num1 operator num2'")
        
        num1_str, operator, num2_str = parts
        
        # Smart type conversion
        num1 = float(num1_str) if '.' in num1_str else int(num1_str)
        num2 = float(num2_str) if '.' in num2_str else int(num2_str)
        
        type_log.append(f"num1: {num1_str} -> {type(num1).__name__}")
        type_log.append(f"num2: {num2_str} -> {type(num2).__name__}")
        
        # Operations mapping
        operations = {
            '+': lambda a, b: a + b,
            '-': lambda a, b: a - b,
            '*': lambda a, b: a * b,
            '/': lambda a, b: a / b if b != 0 else None,
        }
        
        if operator not in operations:
            raise ValueError(f"Unsupported operator: {operator}")
        
        result = operations[operator](num1, num2)
        
        if result is None:
            raise ZeroDivisionError("Division by zero")
        
        type_log.append(f"result: {result} ({type(result).__name__})")
        
        return result, type_log
        
    except (ValueError, ZeroDivisionError) as e:
        type_log.append(f"ERROR: {e}")
        return None, type_log


# Test
expression = "10 + 5.5"
result, log = flexible_calculator(expression)
print(f"Input: \"{expression}\"")
print("Type Log:")
for entry in log:
    print(f"  {entry}")
print(f"Result: {result}")
```
</details>


## 📝 Exercise 3.1: Arithmetic Operations ⭐ (Basic)

**Learning Objective**: Master all numeric operators.

### Challenge:
Define two integers: `a=15`, `b=4`

Calculate and print **all 7 operations**:
1. Addition (`+`)
2. Subtraction (`-`)
3. Multiplication (`*`)
4. Division (`/`)
5. Floor Division (`//`)
6. Modulus (`%`)
7. Exponentiation (`**`)

**Expected Output**:
```
15 + 4 = 19
15 - 4 = 11
15 * 4 = 60
15 / 4 = 3.75
15 // 4 = 3
15 % 4 = 3
15 ** 4 = 50625
```

In [ ]:
# --- Starter (Exercise 3.1) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 3.1.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

a, b = 15, 4

print(f"{a} + {b} = {a + b}")
print(f"{a} - {b} = {a - b}")
print(f"{a} * {b} = {a * b}")
print(f"{a} / {b} = {a / b}")
print(f"{a} // {b} = {a // b}")
print(f"{a} % {b} = {a % b}")
print(f"{a} ** {b} = {a ** b}")
```
</details>


## 📝 Exercise 3.2: Floating-Point Precision ⭐⭐ (Intermediate)

**Learning Objective**: Understand floating-point representation issues.

### Challenge:
Demonstrate numeric precision problems:
1. Add `0.1` **ten times**
2. Compare result to `1.0` (observe precision error)
3. Use `round()` to control precision
4. Include comments explaining **why** issues occur

**Expected Output**:
```
Sum of 0.1 added 10 times: 0.9999999999999999
Equal to 1.0? False
Rounded to 2 decimals: 1.0
```

In [ ]:
# --- Starter (Exercise 3.2) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 3.2.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

# Add 0.1 ten times
total = sum([0.1] * 10)

print(f"Sum of 0.1 added 10 times: {total}")
print(f"Equal to 1.0? {total == 1.0}")

# Fix with rounding
print(f"Rounded to 2 decimals: {round(total, 2)}")

"""
Why this happens:
- Computers store numbers in binary (base-2)
- 0.1 cannot be represented exactly in binary
- Like 1/3 = 0.333... in decimal (infinite)
- Small errors accumulate with repeated operations
- Always use round() for comparisons with floats
"""
```
</details>


## 📝 Exercise 3.3: Physics Calculator ⭐⭐⭐ (Advanced)

**Learning Objective**: Apply numeric operations to real-world formulas.

### Challenge:
Build a physics calculator with two formulas:
1. **Kinetic Energy**: $KE = \frac{1}{2} m v^2$
2. **Relativistic Mass**: $m_{rel} = \frac{m}{\sqrt{1 - \frac{v^2}{c^2}}}$

Handle edge cases:
- Negative mass → error
- Velocity >= speed of light → error or complex number

Use constants: `c = 3e8` (m/s)

**Expected Output**:
```
Object: m=10kg, v=1000m/s
Kinetic Energy: 5000000.0 J
Relativistic Mass: 10.000000055555556 kg
```

In [ ]:
# --- Starter (Exercise 3.3) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 3.3.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

import math

def physics_calculator(mass: float, velocity: float):
    """
    Calculates kinetic energy and relativistic mass.
    
    Args:
        mass (float): Mass in kg
        velocity (float): Velocity in m/s
    
    Returns:
        dict: Results with KE and relativistic mass
    """
    c = 3e8  # Speed of light (m/s)
    
    # Validation
    if mass <= 0:
        raise ValueError("Mass must be positive")
    if velocity >= c:
        raise ValueError(f"Velocity ({velocity}) exceeds speed of light")
    
    # 1. Kinetic Energy
    ke = 0.5 * mass * velocity**2
    
    # 2. Relativistic Mass
    gamma_denominator = 1 - (velocity**2 / c**2)
    m_rel = mass / math.sqrt(gamma_denominator)
    
    return {
        "kinetic_energy": ke,
        "relativistic_mass": m_rel
    }

# Test
m, v = 10, 1000  # kg, m/s
results = physics_calculator(m, v)

print(f"Object: m={m}kg, v={v}m/s")
print(f"Kinetic Energy: {results['kinetic_energy']} J")
print(f"Relativistic Mass: {results['relativistic_mass']} kg")
```
</details>


## 📝 Exercise 4.1: String Methods ⭐ (Basic)

**Learning Objective**: Master common string methods.

### Challenge:
Create string: `"Python Programming"`

Demonstrate these methods:
1. `upper()` - all uppercase
2. `lower()` - all lowercase
3. `strip()` - remove whitespace
4. `split()` - split into words
5. Get **length** using `len()`
6. Get **first 5 characters** using slicing

**Expected Output**:
```
Original: Python Programming
Upper: PYTHON PROGRAMMING
Lower: python programming
Words: ['Python', 'Programming']
Length: 18
First 5: Pytho
```

In [ ]:
# --- Starter (Exercise 4.1) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 4.1.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

text = "Python Programming"

print(f"Original: {text}")
print(f"Upper: {text.upper()}")
print(f"Lower: {text.lower()}")
print(f"Words: {text.split()}")
print(f"Length: {len(text)}")
print(f"First 5: {text[:5]}")
```
</details>


## 📝 Exercise 4.2: Resume Formatter ⭐⭐ (Intermediate)

**Learning Objective**: Advanced string formatting and alignment.

### Challenge:
Create a resume formatter that takes:
- Input: `name`, `role`, `years`, `skills` (list)
- Output: Formatted resume using:
  - f-strings
  - String methods (`capitalize()`, `title()`, `replace()`)
  - Padding/alignment

**Expected Output**:
```
===========================
    JOHN SMITH (5 Years)
Role: Software Engineer
Skills: Python, C++, SQL
===========================
```

In [ ]:
# --- Starter (Exercise 4.2) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 4.2.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

def format_resume(name: str, role: str, years: int, skills: list):
    """
    Formats a professional resume display.
    """
    # Format name: all caps, centered
    name_formatted = name.upper().center(27)
    
    # Format role: title case
    role_formatted = role.title()
    
    # Format skills: join with commas
    skills_formatted = ", ".join(skills)
    
    resume = f"""===========================
{name_formatted}
({years} Years)
Role: {role_formatted}
Skills: {skills_formatted}
==========================="""
    
    return resume

# Test
result = format_resume(
    name="john smith",
    role="software engineer",
    years=5,
    skills=["Python", "C++", "SQL"]
)

print(result)
```
</details>


## 📝 Exercise 4.3: Text Analyzer ⭐⭐⭐ (Advanced)

**Goal**: combine string cleanup + counting + dictionaries.

### Task
Implement `analyze_text(text)` that returns a dictionary with:
- `cleaned`: lowercased text with punctuation removed and whitespace normalized
- `word_count`: number of words in `cleaned`
- `char_count`: number of non-space characters in `cleaned`
- `top_word`: the most frequent word, or `None` if there are no words

### Cleaning rules
- Treat the characters `. , ! ?` as punctuation to remove (replace with spaces).
- Lowercase everything.
- Collapse multiple spaces into a single space, and strip ends.

### Example
```python
analyze_text('Hello, hello   world!')
# {'cleaned': 'hello hello world', 'word_count': 3, 'char_count': 15, 'top_word': 'hello'}
```

In [ ]:
# --- Starter (Exercise 4.3) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here (don’t scroll to the solution cell until you try!).

def analyze_text(text: str) -> dict[str, object]:
    # TODO: implement
    raise NotImplementedError


# Optional quick checks (uncomment after implementing)
# out = analyze_text("Hello, hello   world!")
# assert out["cleaned"] == "hello hello world"
# assert out["word_count"] == 3
# assert out["top_word"] in {"hello", "world"}
# assert analyze_text("   !!! ")["top_word"] is None
# print("✅ 4.3 OK")


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Replace punctuation with spaces, then normalize whitespace with: `" ".join(text.lower().split())`
- Build a frequency dict with `counts.get(word, 0) + 1`
- `top_word` can be: `max(counts, key=counts.get)` (when the dict is not empty)

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

def analyze_text(text: str) -> dict[str, object]:
    if text is None:
        text = ""

    normalized = text
    for ch in ".,!?":
        normalized = normalized.replace(ch, " ")

    cleaned = " ".join(normalized.lower().split())
    words = cleaned.split() if cleaned else []

    char_count = sum(1 for ch in cleaned if ch != " ")

    counts: dict[str, int] = {}
    for w in words:
        counts[w] = counts.get(w, 0) + 1

    top_word = None
    if counts:
        top_word = max(counts, key=counts.get)

    return {
        "cleaned": cleaned,
        "word_count": len(words),
        "char_count": char_count,
        "top_word": top_word,
    }


# Quick checks
out = analyze_text("Hello, hello   world!")
assert out["cleaned"] == "hello hello world"
assert out["word_count"] == 3
assert out["top_word"] in {"hello", "world"}
assert analyze_text("   !!! ")["top_word"] is None
print("✅ 4.3 OK")
```
</details>


## 📝 Exercise 5.1: Boolean Logic ⭐ (Basic)

**Learning Objective**: Understand comparison and logical operators.

### Challenge:
Create variables: `a=10`, `b=20`

Evaluate and print:
1. `a < b` (less than)
2. `a == b` (equality)
3. `not a > b` (negation)
4. `(a < b) and (b > 15)` (AND)
5. `(a > 20) or (b == 20)` (OR)

**Expected Output**:
```
a < b: True
a == b: False
not a > b: True
(a < b) and (b > 15): True
(a > 20) or (b == 20): True
```

In [ ]:
# --- Starter (Exercise 5.1) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 5.1.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

a, b = 10, 20

print(f"a < b: {a < b}")
print(f"a == b: {a == b}")
print(f"not a > b: {not a > b}")
print(f"(a < b) and (b > 15): {(a < b) and (b > 15)}")
print(f"(a > 20) or (b == 20): {(a > 20) or (b == 20)}")
```
</details>


## 📝 Exercise 5.2: Grade Assigner ⭐⭐ (Intermediate)

**Learning Objective**: Apply conditional logic with ternary operators.

### Challenge:
Create a grade assignment function:
- Input: `score` (0-100)
- Logic:
  - A: 90+
  - B: 80-89
  - C: 70-79
  - F: <70
- Use ternary operator: `value if condition else other_value`
- Validate input (0-100 range)

**Expected Output**:
```
Score: 95 → Grade: A
Score: 83 → Grade: B
Score: 65 → Grade: F
Score: 150 → Error: Invalid score
```

In [ ]:
# --- Starter (Exercise 5.2) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 5.2.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

def assign_grade(score: int) -> str:
    """
    Assigns letter grade based on numeric score.
    Uses chained ternary operators.
    """
    # Validation
    if not (0 <= score <= 100):
        return "Error: Invalid score"
    
    # Ternary chain
    grade = ('A' if score >= 90 else
             'B' if score >= 80 else
             'C' if score >= 70 else
             'F')
    
    return grade

# Test cases
test_scores = [95, 83, 65, 150]

for score in test_scores:
    grade = assign_grade(score)
    print(f"Score: {score} → Grade: {grade}")
```
</details>


## 📝 Exercise 5.3: Logic Gate Simulator ⭐⭐⭐ (Advanced)

**Learning Objective**: Implement boolean algebra and truth tables.

### Challenge:
Simulate logic circuits:
1. Implement gates as functions:
   - `NOT(a)`
   - `AND(a, b)`
   - `OR(a, b)`
   - `XOR(a, b)` (exclusive OR)
2. Build **majority gate**: `majority(a, b, c)` - returns True if ≥2 inputs are True
3. Test all 8 possible inputs ($2^3$)
4. Print truth table

**Expected Output**:
```
Truth Table: Majority Gate
| A | B | C | Output |
|---|---|---|--------|
| 0 | 0 | 0 |   0    |
| 0 | 0 | 1 |   0    |
| 0 | 1 | 0 |   0    |
| 0 | 1 | 1 |   1    |
...
```

In [ ]:
# --- Starter (Exercise 5.3) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 5.3.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

def NOT(a: bool) -> bool:
    """NOT gate: Returns opposite of input."""
    return not a

def AND(a: bool, b: bool) -> bool:
    """AND gate: True if both inputs are True."""
    return a and b

def OR(a: bool, b: bool) -> bool:
    """OR gate: True if any input is True."""
    return a or b

def XOR(a: bool, b: bool) -> bool:
    """XOR gate: True if inputs differ."""
    return (a or b) and not (a and b)

def majority(a: bool, b: bool, c: bool) -> bool:
    """
    Majority gate: Returns True if ≥2 inputs are True.
    Logic: (a AND b) OR (b AND c) OR (a AND c)
    """
    return (a and b) or (b and c) or (a and c)


# Truth Table
print("Truth Table: Majority Gate")
print("| A | B | C | Output |")
print("|---|---|---|--------|")

for a in [False, True]:
    for b in [False, True]:
        for c in [False, True]:
            result = majority(a, b, c)
            # Convert bool to 0/1 for display
            a_int, b_int, c_int, r_int = int(a), int(b), int(c), int(result)
            print(f"| {a_int} | {b_int} | {c_int} |   {r_int}    |")
```
</details>


## Exercise 6.1: List Operations ⭐ (Basic)

**Challenge**: Create a list manipulation function that demonstrates core list operations.

**Task**: Write a function that takes a list of integers and:
- Appends the sum of all elements to the end
- Removes the first element
- Inserts the value 100 at index 2
- Returns the modified list and its length

**Constraints**:
- Use built-in list methods (append, remove, insert)
- Handle empty lists gracefully
- Return tuple: (modified_list, length)

**Example**:
```python
Input: [10, 20, 30, 40]
Output: ([20, 30, 100, 40, 100], 5)
```

In [ ]:
# --- Starter (Exercise 6.1) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 6.1.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


In [ ]:
def manipulate_list(numbers: list[int]) -> tuple[list[int], int]:
    """
    Demonstrates core list operations: append, remove, insert.
    
    Performs a series of list manipulations:
    1. Calculates sum and appends it
    2. Removes first element
    3. Inserts 100 at index 2
    
    Args:
        numbers (list[int]): Input list of integers
    
    Returns:
        tuple[list[int], int]: Modified list and its new length
    
    Time Complexity: O(n) - iterating for sum
    Space Complexity: O(1) - in-place modifications
    """
    # Handle empty list edge case
    if not numbers:
        return ([], 0)
    
    # Step 1: Calculate sum and append
    total = sum(numbers)
    numbers.append(total)
    
    # Step 2: Remove first element (if list has elements)
    if numbers:
        numbers.pop(0)  # or numbers.remove(numbers[0])
    
    # Step 3: Insert 100 at index 2 (if possible)
    if len(numbers) >= 2:
        numbers.insert(2, 100)
    
    return (numbers, len(numbers))


# Test cases
print("Exercise 6.1: List Operations")
print("-" * 40)

# Test 1: Normal case
result, length = manipulate_list([10, 20, 30, 40])
print(f"Input: [10, 20, 30, 40]")
print(f"Output: {result}, Length: {length}")
print(f"Expected: [20, 30, 100, 40, 100], Length: 5")

# Test 2: Empty list
result2, length2 = manipulate_list([])
print(f"\nEmpty list: {result2}, Length: {length2}")

# Test 3: Single element
result3, length3 = manipulate_list([5])
print(f"Single element [5]: {result3}, Length: {length3}")

## Exercise 6.2: List Comprehension ⭐⭐ (Intermediate)

**Challenge**: Use list comprehensions to filter and transform data efficiently.

**Task**: Given a list of mixed positive/negative integers, create a function that returns:
1. A list of squares of only the positive even numbers
2. A list of absolute values of negative numbers
3. Count of numbers divisible by 3

**Constraints**:
- Use list comprehensions for all operations
- Single pass through the list
- No loops (for/while) allowed in main logic

**Example**:
```python
Input: [2, -3, 4, -5, 6, 9, -12]
Output: (
    [4, 16, 36],      # Squares of positive evens: 2², 4², 6²
    [3, 5, 12],       # Absolute values of negatives
    3                 # Count divisible by 3: -3, 6, 9
)
```

In [ ]:
# --- Starter (Exercise 6.2) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 6.2.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


In [ ]:
def analyze_numbers(numbers: list[int]) -> tuple[list[int], list[int], int]:
    """
    Demonstrates list comprehensions for filtering and transformation.
    
    Uses concise list comprehension syntax to:
    1. Filter and square positive even numbers
    2. Extract absolute values of negatives
    3. Count numbers divisible by 3
    
    Args:
        numbers (list[int]): Mixed positive/negative integers
    
    Returns:
        tuple: (positive_even_squares, abs_negatives, div_by_3_count)
    
    Time Complexity: O(n) - three passes through list
    Space Complexity: O(n) - storing filtered results
    """
    # List comprehension: square of positive even numbers
    positive_even_squares = [x**2 for x in numbers if x > 0 and x % 2 == 0]
    
    # List comprehension: absolute values of negative numbers
    abs_negatives = [abs(x) for x in numbers if x < 0]
    
    # Count divisible by 3 (using sum with generator expression)
    div_by_3_count = sum(1 for x in numbers if x % 3 == 0)
    
    return (positive_even_squares, abs_negatives, div_by_3_count)


# Test cases
print("Exercise 6.2: List Comprehension")
print("-" * 40)

# Test 1: Mixed numbers
test_data = [2, -3, 4, -5, 6, 9, -12]
squares, negatives, count = analyze_numbers(test_data)
print(f"Input: {test_data}")
print(f"Positive even squares: {squares}")
print(f"Absolute negatives: {negatives}")
print(f"Divisible by 3 count: {count}")

# Test 2: All positive
test_data2 = [2, 4, 6, 8]
result2 = analyze_numbers(test_data2)
print(f"\nAll positive {test_data2}: {result2}")

# Test 3: All negative
test_data3 = [-1, -2, -3, -6]
result3 = analyze_numbers(test_data3)
print(f"All negative {test_data3}: {result3}")

## 📝 Exercise 6.3: Moving Average ⭐⭐⭐ (Advanced)

**Goal**: practice list slicing + loops + edge cases.

### Task
Implement `moving_average(values, window)` using a sliding window.

Return a list where each element is the average of a consecutive window of size `window`.

### Rules
- If `window <= 0`, raise `ValueError`.
- If `window > len(values)`, return an empty list `[]`.
- Output values should be floats.

### Example
```python
moving_average([1, 2, 3, 4], 2)
# [1.5, 2.5, 3.5]
```

In [ ]:
# --- Starter (Exercise 6.3) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here (don’t scroll to the solution cell until you try!).

def moving_average(values: list[float], window: int) -> list[float]:
    # TODO: implement
    raise NotImplementedError


# Optional quick checks (uncomment after implementing)
# assert moving_average([1, 2, 3, 4], 2) == [1.5, 2.5, 3.5]
# assert moving_average([10, 20, 30], 3) == [20.0]
# assert moving_average([1, 2], 3) == []
# print("✅ 6.3 OK")


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Validate `window >= 1` (otherwise raise `ValueError`).
- Use slicing: `values[i : i + window]`
- Number of windows is `len(values) - window + 1`
- Average is `sum(chunk) / window`

</details>


<details>
<summary>✅ Solution (Instructor only)</summary>

```python

def moving_average(values: list[float], window: int) -> list[float]:
    if window <= 0:
        raise ValueError("window must be >= 1")
    if window > len(values):
        return []

    out: list[float] = []
    for i in range(len(values) - window + 1):
        chunk = values[i : i + window]
        out.append(sum(chunk) / window)
    return out


# Quick checks
assert moving_average([1, 2, 3, 4], 2) == [1.5, 2.5, 3.5]
assert moving_average([10, 20, 30], 3) == [20.0]
assert moving_average([1, 2], 3) == []
print("✅ 6.3 OK")
```
</details>


## Exercise 7.1: Tuple Basics ⭐ (Basic)

**Challenge**: Understand tuple immutability and basic operations.

**Task**: Create a function that takes employee data as a tuple `(name, age, salary)` and:
- Validates that all fields are present
- Returns a formatted string with the data
- Attempts to demonstrate tuple immutability
- Returns tuple length and type of each element

**Constraints**:
- Do not convert tuple to list
- Use tuple indexing and unpacking
- Return tuple of results

**Example**:
```python
Input: ("Alice", 30, 75000)
Output: (
    "Employee: Alice, Age: 30, Salary: $75000",
    3,
    ['str', 'int', 'int']
)
```

In [ ]:
# --- Starter (Exercise 7.1) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 7.1.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


In [ ]:
def analyze_employee_tuple(employee: tuple) -> tuple[str, int, list[str]]:
    """
    Demonstrates tuple immutability and basic tuple operations.
    
    Tuples are immutable sequences - once created, their elements
    cannot be modified, added, or removed. They're perfect for
    storing fixed collections of heterogeneous data.
    
    Args:
        employee (tuple): Employee data (name, age, salary)
    
    Returns:
        tuple: (formatted_string, length, types_list)
    
    Time Complexity: O(n) where n is tuple length
    Space Complexity: O(n) for types list
    """
    # Validate tuple has 3 elements
    if len(employee) != 3:
        return ("Invalid employee data", 0, [])
    
    # Tuple unpacking (elegant way to extract values)
    name, age, salary = employee
    
    # Format string using tuple indexing
    formatted = f"Employee: {employee[0]}, Age: {employee[1]}, Salary: ${employee[2]}"
    
    # Get types of each element
    types_list = [type(element).__name__ for element in employee]
    
    # Demonstrate immutability (this would raise TypeError if uncommented)
    # employee[0] = "Bob"  # TypeError: 'tuple' object does not support item assignment
    
    return (formatted, len(employee), types_list)


# Test cases
print("Exercise 7.1: Tuple Basics")
print("-" * 40)

# Test 1: Valid employee
employee1 = ("Alice", 30, 75000)
formatted, length, types = analyze_employee_tuple(employee1)
print(f"Input: {employee1}")
print(f"Formatted: {formatted}")
print(f"Length: {length}")
print(f"Types: {types}")

# Test 2: Different employee
employee2 = ("Bob", 45, 120000)
result2 = analyze_employee_tuple(employee2)
print(f"\nEmployee 2: {result2[0]}")

# Demonstrate tuple immutability
print("\n🔒 Tuples are immutable:")
print(f"Original: {employee1}")
# employee1[0] = "Charlie"  # This would raise TypeError
print("Cannot modify tuple elements directly!")

## Exercise 7.2: Tuple Unpacking ⭐⭐ (Intermediate)

**Challenge**: Master advanced tuple unpacking techniques.

**Task**: Create a function that processes a nested tuple of coordinates and:
- Unpacks coordinates using `*` (star) operator for variable-length unpacking
- Swaps first and last coordinates
- Calculates the distance between first two points
- Returns unpacked values in different formats

**Constraints**:
- Use multiple assignment (a, b = tuple)
- Demonstrate `*rest` unpacking pattern
- Use tuple unpacking in function parameters

**Example**:
```python
Input: ((0, 0), (3, 4), (6, 8), (10, 10))
Output: {
    'first': (0, 0),
    'last': (10, 10),
    'middle': [(3, 4), (6, 8)],
    'distance': 5.0,  # Between (0,0) and (3,4)
    'swapped': ((10, 10), (3, 4), (6, 8), (0, 0))
}
```

In [ ]:
# --- Starter (Exercise 7.2) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 7.2.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


In [ ]:
import math

def process_coordinates(coords: tuple) -> dict:
    """
    Demonstrates advanced tuple unpacking with * operator and swapping.
    
    Tuple unpacking allows elegant extraction of values:
    - Basic: a, b = (1, 2)
    - Star: first, *rest, last = (1, 2, 3, 4, 5)
    - Nested: (x, y), (a, b) = ((1, 2), (3, 4))
    
    Args:
        coords (tuple): Nested tuple of (x, y) coordinate pairs
    
    Returns:
        dict: Dictionary with unpacked and processed values
    
    Time Complexity: O(n) for unpacking and distance calculation
    Space Complexity: O(n) for storing middle coordinates
    """
    # Advanced unpacking: first, middle elements, last
    first, *middle, last = coords
    
    # Unpack first two coordinates for distance calculation
    if len(coords) >= 2:
        (x1, y1), (x2, y2) = coords[0], coords[1]
        distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    else:
        distance = 0.0
    
    # Swap first and last using tuple unpacking
    swapped = (last, *middle, first)
    
    return {
        'first': first,
        'last': last,
        'middle': list(middle),
        'distance': round(distance, 2),
        'swapped': swapped
    }


# Test cases
print("Exercise 7.2: Tuple Unpacking")
print("-" * 40)

# Test 1: Multiple coordinates
coords1 = ((0, 0), (3, 4), (6, 8), (10, 10))
result = process_coordinates(coords1)
print(f"Input: {coords1}")
print(f"First: {result['first']}")
print(f"Last: {result['last']}")
print(f"Middle: {result['middle']}")
print(f"Distance (first two): {result['distance']}")
print(f"Swapped: {result['swapped']}")

# Test 2: Minimal coordinates
coords2 = ((1, 1), (4, 5))
result2 = process_coordinates(coords2)
print(f"\nMinimal {coords2}:")
print(f"Distance: {result2['distance']}")

# Demonstrate unpacking patterns
print("\n🎯 Unpacking Examples:")
a, b, c = (10, 20, 30)
print(f"Basic: a={a}, b={b}, c={c}")

first, *rest = (1, 2, 3, 4, 5)
print(f"Star: first={first}, rest={rest}")

## Exercise 7.3: Named Tuples ⭐⭐⭐ (Advanced)

**Challenge**: Use `namedtuple` from `collections` for self-documenting data structures.

**Task**: Create a student record system using `namedtuple` that:
- Defines a `Student` named tuple with fields: (id, name, grades, gpa)
- Creates multiple student records
- Implements a function to calculate GPA from grades
- Demonstrates the benefits: field access by name, immutability, memory efficiency

**Constraints**:
- Use `collections.namedtuple`
- Access fields by name (dot notation)
- Show conversion to dict
- Compare with regular tuple and dict memory usage

**Example**:
```python
Student = namedtuple('Student', ['id', 'name', 'grades', 'gpa'])
student = Student(101, 'Alice', [85, 90, 88], 3.7)
Output: 
    student.name → 'Alice'
    student.gpa → 3.7
    student._asdict() → {'id': 101, 'name': 'Alice', ...}
```

In [ ]:
# --- Starter (Exercise 7.3) ---

# --- Progress (auto) ---
try:
    if 'game' in globals():
        print(game.render_text_dashboard())
except Exception:
    pass

# Write your solution here first.

# TODO: your code


### 💡 Hint (click to expand)
<details>
<summary>Show hint</summary>

- Re-read the prompt for Exercise 7.3.
- Start small, then add edge-case handling.
- Use 1–3 `assert` checks to self-test.

</details>


In [ ]:
from collections import namedtuple
import sys

def create_student_system():
    """
    Demonstrates namedtuple for creating self-documenting, immutable records.
    
    Benefits of namedtuple over regular tuples:
    1. Fields accessible by name (student.name vs student[1])
    2. Self-documenting code (clear field names)
    3. Immutable like tuples (thread-safe, hashable)
    4. Less memory than dicts (no per-instance __dict__)
    5. Can be used as dict keys (unlike mutable dicts)
    
    Returns:
        dict: Student records and analysis
    
    Time Complexity: O(n*m) where n=students, m=grades per student
    Space Complexity: O(n) for storing records
    """
    # Define Student namedtuple structure
    Student = namedtuple('Student', ['id', 'name', 'grades', 'gpa'])
    
    # Helper function to calculate GPA from grades
    def calculate_gpa(grades: list[int]) -> float:
        """Convert percentage grades to 4.0 GPA scale"""
        avg = sum(grades) / len(grades) if grades else 0
        if avg >= 90: return 4.0
        elif avg >= 80: return 3.0
        elif avg >= 70: return 2.0
        elif avg >= 60: return 1.0
        else: return 0.0
    
    # Create student records
    students = [
        Student(101, 'Alice', [85, 90, 88], calculate_gpa([85, 90, 88])),
        Student(102, 'Bob', [92, 95, 89], calculate_gpa([92, 95, 89])),
        Student(103, 'Charlie', [78, 82, 80], calculate_gpa([78, 82, 80]))
    ]
    
    # Demonstrate namedtuple features
    alice = students[0]
    
    print("Exercise 7.3: Named Tuples")
    print("=" * 50)
    
    # 1. Access by name (readable!)
    print(f"✅ Access by name: {alice.name} (ID: {alice.id})")
    print(f"   GPA: {alice.gpa}")
    
    # 2. Convert to dict
    print(f"\n📋 As dict: {alice._asdict()}")
    
    # 3. Immutability
    print(f"\n🔒 Immutable (like tuple):")
    try:
        alice.gpa = 4.0  # This will raise AttributeError
    except AttributeError as e:
        print(f"   Cannot modify: {e}")
    
    # 4. Memory comparison
    regular_tuple = (101, 'Alice', [85, 90, 88], 3.0)
    regular_dict = {'id': 101, 'name': 'Alice', 'grades': [85, 90, 88], 'gpa': 3.0}
    
    print(f"\n💾 Memory Usage Comparison:")
    print(f"   namedtuple: {sys.getsizeof(alice)} bytes")
    print(f"   tuple:      {sys.getsizeof(regular_tuple)} bytes")
    print(f"   dict:       {sys.getsizeof(regular_dict)} bytes")
    
    # 5. Can be used as dict keys (hashable)
    student_lookup = {alice: "Top performer"}
    print(f"\n🔑 Hashable (can be dict key): {hash(alice)}")
    
    # 6. Replace values (creates new instance)
    alice_updated = alice._replace(gpa=4.0)
    print(f"\n🔄 _replace() creates new instance:")
    print(f"   Original: {alice.gpa}")
    print(f"   Updated:  {alice_updated.gpa}")
    
    return {
        'students': students,
        'total': len(students),
        'avg_gpa': sum(s.gpa for s in students) / len(students)
    }

# Run the demonstration
result = create_student_system()
print(f"\n📊 Summary: {result['total']} students, Avg GPA: {result['avg_gpa']:.2f}")

---
# 🚀 Next Phases (Phase 2+ Continued)
Everything from here onward remains as originally authored.
---


---
## 📦 Phase 2 (Continued): Sets & Dictionaries
These exercises build on Lists/Tuples and introduce **Sets** (unique items) and **Dictionaries** (key → value storage).

### Exercises in this section
- **8.1 Set Operations** ⭐
- **8.2 Duplicate Remover** ⭐⭐
- **9.1 Dictionary Basics (Word Count)** ⭐
- **9.2 Config Merge** ⭐⭐
- **9.3 Nested Dict Explorer** ⭐⭐⭐
---

In [ ]:
# ================================
# Exercises 8.1 to 9.3 (Starter)
# ================================

from __future__ import annotations

from typing import Any, Iterable


# --- Exercise 8.1: Set Operations ⭐ ---
def analyze_enrollment(class_a: Iterable[Any], class_b: Iterable[Any]) -> dict[str, set[Any]]:
    """
    Compute set operations for two classes.

    Return a dict with:
    - 'union': all unique students
    - 'intersection': students in both classes
    - 'only_a': only in class_a
    - 'only_b': only in class_b
    """
    # TODO: Implement using set() and set operators | & -
    a_set = set(class_a)
    b_set = set(class_b)
    return {
        "union": a_set | b_set,
        "intersection": a_set & b_set,
        "only_a": a_set - b_set,
        "only_b": b_set - a_set,
    }


print("Exercise 8.1 quick test")
result_8_1 = analyze_enrollment([101, 102, 103, 104], [103, 104, 105])
assert result_8_1["intersection"] == {103, 104}
assert result_8_1["only_b"] == {105}
print("✅ 8.1 OK")


# --- Exercise 8.2: Duplicate Remover ⭐⭐ ---
def dedupe_preserve_order(items: Iterable[Any]) -> tuple[list[Any], int]:
    """
    Remove duplicates but keep the first-seen order.

    Returns: (unique_items, duplicates_removed)

    Assumption: items are hashable (so we can track seen items in a set).
    """
    # TODO: Implement using a set() called 'seen'
    seen: set[Any] = set()
    unique: list[Any] = []
    removed = 0
    for item in items:
        if item in seen:
            removed += 1
            continue
        seen.add(item)
        unique.append(item)
    return unique, removed


print("Exercise 8.2 quick test")
unique_8_2, removed_8_2 = dedupe_preserve_order(["a", "b", "a", "c", "b", "d"])
assert unique_8_2 == ["a", "b", "c", "d"]
assert removed_8_2 == 2
print("✅ 8.2 OK")


# --- Exercise 9.1: Dictionary Basics (Word Count) ⭐ ---
def word_frequency(text: str) -> dict[str, int]:
    """Count how many times each word appears (case-insensitive)."""
    # TODO: Normalize to lower-case and split; use dict.get() to count
    separators = "\n\t.,!?:;\"'()[]{}<>/\\|@#$%^&*-_=+~`"
    cleaned = text.lower()
    for ch in separators:
        cleaned = cleaned.replace(ch, " ")
    counts: dict[str, int] = {}
    for word in cleaned.split():
        counts[word] = counts.get(word, 0) + 1
    return counts


print("Exercise 9.1 quick test")
counts_9_1 = word_frequency("Hello hello, world!")
assert counts_9_1 == {"hello": 2, "world": 1}
print("✅ 9.1 OK")


# --- Exercise 9.2: Config Merge ⭐⭐ ---
def merge_configs(base: dict[str, Any], override: dict[str, Any]) -> dict[str, Any]:
    """Return a new dict where override values replace base values."""
    # TODO: Implement shallow merge without modifying inputs
    merged = dict(base)
    merged.update(override)
    return merged


print("Exercise 9.2 quick test")
merged_9_2 = merge_configs({"theme": "dark", "font": 12}, {"font": 14})
assert merged_9_2 == {"theme": "dark", "font": 14}
print("✅ 9.2 OK")


# --- Exercise 9.3: Nested Dict Explorer ⭐⭐⭐ ---
def deep_get(data: dict[str, Any], path: str, default: Any | None = None) -> Any:
    """Safely fetch a nested value using a dot-path like 'user.profile.name'."""
    # TODO: Walk the dict key-by-key; return default if missing
    current: Any = data
    for key in path.split("."):
        if not isinstance(current, dict):
            return default
        if key not in current:
            return default
        current = current[key]
    return current


print("Exercise 9.3 quick test")
data_9_3 = {"user": {"profile": {"name": "Asha"}}}
assert deep_get(data_9_3, "user.profile.name") == "Asha"
assert deep_get(data_9_3, "user.profile.age", default=0) == 0
print("✅ 9.3 OK")

---
## 🧠 Phase 2 (Continued): Operators & Control Flow
Focus: operator precedence, comparisons, and decision making with `if/elif/else` and `match/case`.

### Exercises in this section
- **10.1 Operator Precedence** ⭐
- **10.2 Age Classifier** ⭐⭐
- **11.1 Traffic Light Controller** ⭐
- **11.2 Match-Case Command Parser** ⭐⭐
- **11.3 FizzBuzz Challenge** ⭐⭐
---

In [ ]:
# ======================================
# Exercises 10.1 to 11.3 (Starter)
# ======================================

from __future__ import annotations

from dataclasses import dataclass

# --- Exercise 10.1: Operator Precedence ⭐ ---
def precedence_demo(a: int, b: int, c: int) -> dict[str, int]:
    """Return results of similar expressions to show precedence."""
    # TODO: Complete the dict. Hint: ** has higher precedence than * than +
    return {
        "a_plus_b_times_c": a + b * c,
        "(a_plus_b)_times_c": (a + b) * c,
        "a_pow_b_pow_c": a ** b ** c,  # exponentiation is right-associative
    }


print("10.1 quick test")
d = precedence_demo(2, 3, 4)
assert d["a_plus_b_times_c"] == 14  # 2 + (3*4)
assert d["(a_plus_b)_times_c"] == 20  # (2+3)*4
print("✅ 10.1 OK")


# --- Exercise 10.2: Age Classifier ⭐⭐ ---
def age_classifier(age: int) -> str:
    """Return one of: invalid, child, teen, adult, senior."""
    # TODO: Use chained comparisons and if/elif
    if age < 0:
        return "invalid"
    if age <= 12:
        return "child"
    if age <= 19:
        return "teen"
    if age <= 64:
        return "adult"
    return "senior"


print("10.2 quick test")
assert age_classifier(5) == "child"
assert age_classifier(17) == "teen"
assert age_classifier(30) == "adult"
assert age_classifier(80) == "senior"
print("✅ 10.2 OK")


# --- Exercise 11.1: Traffic Light Controller ⭐ ---
def traffic_light(color: str) -> str:
    """Map color to action: red->stop, yellow->slow, green->go."""
    c = color.strip().lower()
    if c == "red":
        return "stop"
    if c == "yellow":
        return "slow"
    if c == "green":
        return "go"
    return "invalid"


print("11.1 quick test")
assert traffic_light("Red") == "stop"
assert traffic_light(" green ") == "go"
assert traffic_light("blue") == "invalid"
print("✅ 11.1 OK")


# --- Exercise 11.2: Match-Case Command Parser ⭐⭐ ---
@dataclass(frozen=True)
class CommandResult:
    ok: bool
    message: str


def parse_command(command: str) -> CommandResult:
    """Parse a small command set: help, status, quit/exit."""
    cmd = command.strip().lower()
    # TODO: Use match/case (Python 3.10+)
    match cmd:
        case "help":
            return CommandResult(True, "commands: help, status, quit")
        case "status":
            return CommandResult(True, "status: ready")
        case "quit" | "exit":
            return CommandResult(True, "bye")
        case _:
            return CommandResult(False, "unknown command")


print("11.2 quick test")
assert parse_command("HELP").ok is True
assert parse_command("status").message.startswith("status")
assert parse_command("nope").ok is False
print("✅ 11.2 OK")


# --- Exercise 11.3: FizzBuzz Challenge ⭐⭐ ---
def fizzbuzz(n: int) -> list[str]:
    """Return FizzBuzz list from 1..n."""
    # TODO: Implement using if/elif/else
    if n < 1:
        return []
    out: list[str] = []
    for i in range(1, n + 1):
        if i % 15 == 0:
            out.append("FizzBuzz")
        elif i % 3 == 0:
            out.append("Fizz")
        elif i % 5 == 0:
            out.append("Buzz")
        else:
            out.append(str(i))
    return out


print("11.3 quick test")
assert fizzbuzz(5) == ["1", "2", "Fizz", "4", "Buzz"]
print("✅ 11.3 OK")

---
## 🔁 Phase 2 (Continued): Loops, Functions, and `range()`
You’ll practice repeating work with loops, writing reusable code with functions, and using `range()` efficiently.

### Exercises in this section
- **12.1 Loop Basics (Sum 1..n)** ⭐
- **12.2 Nested Loops (Multiplication Table)** ⭐⭐
- **12.3 Loop Optimization (Prime Sieve)** ⭐⭐⭐
- **13.1 Function Basics (Converter)** ⭐
- **13.2 Higher-Order Functions (Compose)** ⭐⭐
- **13.3 Decorators (Timing)** ⭐⭐⭐
- **14.1 `range()` Basics (Evens)** ⭐
- **14.2 `range()` vs `list` (Memory)** ⭐⭐
- **14.3 Custom Iterator** ⭐⭐⭐
---

In [ ]:
# ======================================
# Exercises 12.1 to 14.3 (Starter)
# ======================================

from __future__ import annotations

import functools
import sys
import time
from collections.abc import Callable, Iterator

# --- Exercise 12.1: Loop Basics (Sum 1..n) ⭐ ---
def sum_1_to_n(n: int) -> int:
    """Return the sum of integers from 1 to n using a loop."""
    # TODO: Implement using for-loop; handle n < 1
    if n < 1:
        return 0
    total = 0
    for i in range(1, n + 1):
        total += i
    return total


assert sum_1_to_n(5) == 15
assert sum_1_to_n(0) == 0
print("✅ 12.1 OK")

# --- Exercise 12.2: Nested Loops (Multiplication Table) ⭐⭐ ---
def multiplication_table(size: int) -> list[list[int]]:
    """Return an NxN multiplication table as a list of rows."""
    # TODO: Implement nested loops; handle size < 1
    if size < 1:
        return []
    table: list[list[int]] = []
    for r in range(1, size + 1):
        row: list[int] = []
        for c in range(1, size + 1):
            row.append(r * c)
        table.append(row)
    return table


assert multiplication_table(3) == [[1, 2, 3], [2, 4, 6], [3, 6, 9]]
print("✅ 12.2 OK")

# --- Exercise 12.3: Prime Sieve ⭐⭐⭐ ---
def primes_up_to(n: int) -> list[int]:
    """Return all primes up to n (inclusive) using a sieve."""
    # TODO: Implement sieve; handle n < 2
    if n < 2:
        return []
    is_prime = [True] * (n + 1)
    is_prime[0] = False
    is_prime[1] = False
    p = 2
    while p * p <= n:
        if is_prime[p]:
            for multiple in range(p * p, n + 1, p):
                is_prime[multiple] = False
        p += 1
    return [i for i in range(2, n + 1) if is_prime[i]]


assert primes_up_to(10) == [2, 3, 5, 7]
print("✅ 12.3 OK")

# --- Exercise 13.1: Function Basics (Converter) ⭐ ---
def celsius_to_fahrenheit(c: float) -> float:
    """Convert Celsius to Fahrenheit."""
    return (c * 9 / 5) + 32


assert celsius_to_fahrenheit(0) == 32
print("✅ 13.1 OK")

# --- Exercise 13.2: Higher-Order Functions (Compose) ⭐⭐ ---
def compose(*funcs: Callable[[int], int]) -> Callable[[int], int]:
    """Compose functions right-to-left."""
    def _composed(x: int) -> int:
        value = x
        for fn in reversed(funcs):
            value = fn(value)
        return value
    return _composed


add1 = lambda x: x + 1
times2 = lambda x: x * 2
f = compose(add1, times2)  # add1(times2(x))
assert f(3) == 7
print("✅ 13.2 OK")

# --- Exercise 13.3: Decorators (Timing) ⭐⭐⭐ ---
def timing(fn: Callable[..., object]) -> Callable[..., tuple[object, float]]:
    """Decorator that returns (result, elapsed_seconds)."""
    @functools.wraps(fn)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = fn(*args, **kwargs)
        elapsed = time.perf_counter() - start
        return result, elapsed
    return wrapper


@timing
def slow_add(a: int, b: int) -> int:
    time.sleep(0.01)
    return a + b


value, elapsed = slow_add(2, 3)
assert value == 5
assert elapsed >= 0
print("✅ 13.3 OK")

# --- Exercise 14.1: range() Basics (Evens) ⭐ ---
def evens_with_range(start: int, stop: int) -> list[int]:
    """Return even numbers in [start, stop) using range."""
    if start >= stop:
        return []
    first = start if start % 2 == 0 else start + 1
    return list(range(first, stop, 2))


assert evens_with_range(3, 10) == [4, 6, 8]
print("✅ 14.1 OK")

# --- Exercise 14.2: range() vs list (Memory) ⭐⭐ ---
n = 1_000_000
r = range(n)
lst = list(range(1000))  # keep the list smaller to avoid huge memory use
print("14.2 demo")
print("range(1_000_000) size:", sys.getsizeof(r), "bytes")
print("list(range(1000)) size:", sys.getsizeof(lst), "bytes")

# --- Exercise 14.3: Custom Iterator ⭐⭐⭐ ---
class Countdown(Iterator[int]):
    """Iterate from start down to 0."""
    def __init__(self, start: int):
        self._current = start
    def __iter__(self) -> "Countdown":
        return self
    def __next__(self) -> int:
        if self._current < 0:
            raise StopIteration
        value = self._current
        self._current -= 1
        return value


assert list(Countdown(3)) == [3, 2, 1, 0]
print("✅ 14.3 OK")

---
## ⚙️ Phase 3: Classes (OOP) & Error Handling
Now you’ll write your own classes and learn to handle errors safely (instead of crashing).

### Exercises in this section
- **15.1 Class Definition (Dog)** ⭐
- **15.2 Inheritance (Animal → Cat/Dog)** ⭐⭐
- **16.1 Try/Except (Safe Division)** ⭐
- **16.2 Multiple Exceptions (Robust Parsing)** ⭐⭐
- **16.3 Custom Exceptions (Bank Account)** ⭐⭐⭐
---

In [ ]:
# ======================================
# Exercises 15.1 to 16.3 (Starter)
# ======================================

from __future__ import annotations

from dataclasses import dataclass

# --- Exercise 15.1: Class Definition (Dog) ⭐ ---
@dataclass
class Dog:
    name: str
    age: int
    def speak(self) -> str:
        return f"{self.name} says: Woof!"
    def birthday(self) -> None:
        self.age += 1


d = Dog("Buddy", 3)
assert d.speak().startswith("Buddy")
d.birthday()
assert d.age == 4
print("✅ 15.1 OK")

# --- Exercise 15.2: Inheritance (Animal → Cat/Dog) ⭐⭐ ---
@dataclass
class Animal:
    name: str
    def speak(self) -> str:
        return "..."


@dataclass
class Cat(Animal):
    def speak(self) -> str:
        return f"{self.name} says: Meow!"


@dataclass
class Dog2(Animal):
    def speak(self) -> str:
        return f"{self.name} says: Woof!"


pets: list[Animal] = [Cat("Mimi"), Dog2("Rex")]
assert [p.speak() for p in pets] == ["Mimi says: Meow!", "Rex says: Woof!"]
print("✅ 15.2 OK")

# --- Exercise 16.1: Try/Except (Safe Division) ⭐ ---
def safe_divide(a: float, b: float) -> float | None:
    """Return a/b, but return None if b is 0."""
    try:
        return a / b
    except ZeroDivisionError:
        return None


assert safe_divide(10, 2) == 5
assert safe_divide(10, 0) is None
print("✅ 16.1 OK")

# --- Exercise 16.2: Multiple Exceptions (Robust Parsing) ⭐⭐ ---
def read_int_from_text(text: str) -> int | None:
    """Try to parse an int from a string; return None if invalid."""
    try:
        return int(text.strip())
    except (ValueError, AttributeError):
        return None


assert read_int_from_text(" 42 ") == 42
assert read_int_from_text("not a number") is None
print("✅ 16.2 OK")

# --- Exercise 16.3: Custom Exceptions (Bank Account) ⭐⭐⭐ ---
class InvalidAccountError(ValueError):
    pass


@dataclass
class BankAccount:
    owner: str
    balance: float = 0.0
    def deposit(self, amount: float) -> None:
        if amount <= 0:
            raise InvalidAccountError("deposit amount must be positive")
        self.balance += amount
    def withdraw(self, amount: float) -> None:
        if amount <= 0:
            raise InvalidAccountError("withdraw amount must be positive")
        if amount > self.balance:
            raise InvalidAccountError("insufficient funds")
        self.balance -= amount


acct = BankAccount("Asha", 100.0)
acct.deposit(50)
assert acct.balance == 150.0
acct.withdraw(25)
assert acct.balance == 125.0
try:
    acct.withdraw(1000)
    raise AssertionError("Expected InvalidAccountError")
except InvalidAccountError:
    pass
print("✅ 16.3 OK")

---
## 🧾 Phase 3 (Continued): String Formatting & Modules
You’ll learn to format text professionally and organize code across files using modules and packages.

### Exercises in this section
- **17.1 String Formatting Basics** ⭐
- **17.2 Advanced Formatting (Table)** ⭐⭐
- **17.3 Mini Template Engine (HTML)** ⭐⭐⭐
- **18.1 Built-in Modules** ⭐
- **18.2 Custom Module (`my_math.py`)** ⭐⭐
- **18.3 Package Structure (`utils/`)** ⭐⭐⭐
---

In [ ]:
# ======================================
# Exercises 17.1 to 18.3 (Starter)
# ======================================

from __future__ import annotations

import datetime
import math
import random

# --- Exercise 17.1: String Formatting Basics ⭐ ---
def format_user_summary(name: str, role: str, score: float) -> str:
    """Return a one-line summary with clean formatting."""
    # TODO: Use .title() and format score to 1 decimal
    return f"Name: {name.title()} | Role: {role.title()} | Score: {score:.1f}"


assert format_user_summary("john", "data engineer", 91.234) == "Name: John | Role: Data Engineer | Score: 91.2"
print("✅ 17.1 OK")

# --- Exercise 17.2: Advanced Formatting (Table) ⭐⭐ ---
def format_table(rows: list[tuple[str, int]]) -> str:
    """Return a two-column aligned table."""
    # TODO: Align left/right using format specifiers
    lines = [f"{'Item':<20} | {'Value':>8}", "-" * 32]
    for label, value in rows:
        lines.append(f"{label:<20} | {value:>8d}")
    return "\n".join(lines)


print(format_table([("Apples", 12), ("Oranges", 3)]))
print("✅ 17.2 OK")

# --- Exercise 17.3: Mini Template Engine (HTML) ⭐⭐⭐ ---
def render_html_list(title: str, items: list[str]) -> str:
    """Return a simple HTML page as a string."""
    # TODO: Build HTML with <h1> and <ul><li>...
    safe_items = [x.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;") for x in items]
    lis = "\n".join(f"  <li>{x}</li>" for x in safe_items)
    return f"""<!doctype html>
<html>
<head><meta charset='utf-8'><title>{title}</title></head>
<body>
<h1>{title}</h1>
<ul>
{lis}
</ul>
</body>
</html>
"""


html = render_html_list("Shopping", ["Milk", "Bread"])
assert "<li>Milk</li>" in html
print("✅ 17.3 OK")

# --- Exercise 18.1: Built-in Modules ⭐ ---
today = datetime.date.today()
circle_area = math.pi * (3 ** 2)
random.seed(1)
roll = random.randint(1, 6)
print("18.1 demo")
print("Today:", today)
print("Area of circle r=3:", round(circle_area, 2))
print("Dice roll:", roll)
print("✅ 18.1 OK")

# --- Exercise 18.2: Custom Module (my_math.py) ⭐⭐ ---
# NOTE: This repository includes my_math.py at the workspace root.
# NOTE (single-file mode): Instead of importing my_math.py, we define the needed functions inline.

def clamp(x: float, low: float, high: float) -> float:
    return max(low, min(high, x))

def mean(values: list[float]) -> float:
    if not values:
        raise ValueError('mean() requires at least one value')
    return sum(values) / len(values)

assert clamp(10, 0, 5) == 5
assert round(mean([1.0, 2.0, 3.0]), 2) == 2.0
print("✅ 18.2 OK")

# --- Exercise 18.3: Package Structure (utils/) ⭐⭐⭐ ---
# NOTE: This repository includes a utils/ package with converters.py
# NOTE (single-file mode): Instead of importing utils/, we define converters inline.

def km_to_miles(km: float) -> float:
    return float(km) * 0.621371

def miles_to_km(miles: float) -> float:
    return float(miles) / 0.621371

assert round(km_to_miles(10), 3) == 6.214
assert round(miles_to_km(6.21371), 3) == 10.0
print("✅ 18.3 OK")

---
## 🧬 Phase 3 (Continued): Generators & Iterators
Generators help you process data **lazily** (one item at a time) without storing everything in memory.

### Exercises in this section
- **19.1 Countdown Generator** ⭐⭐
- **19.2 Fibonacci Iterator (Iterator Protocol)** ⭐⭐⭐
- **19.3 Generator Expressions (Lazy Pipeline)** ⭐⭐
---

In [ ]:
# ======================================
# Exercises 19.1 to 19.3 (Starter)
# ======================================

from __future__ import annotations

from collections.abc import Iterator

# --- Exercise 19.1: Countdown Generator ⭐⭐ ---
def countdown(start: int):
    """Yield start, start-1, ..., 0."""
    # TODO: Use 'yield' inside a while-loop
    current = start
    while current >= 0:
        yield current
        current -= 1


assert list(countdown(3)) == [3, 2, 1, 0]
print("✅ 19.1 OK")

# --- Exercise 19.2: Fibonacci Iterator ⭐⭐⭐ ---
class Fibonacci(Iterator[int]):
    """Iterator that yields the first n Fibonacci numbers."""
    def __init__(self, n: int):
        self._remaining = max(0, n)
        self._a = 0
        self._b = 1
    def __iter__(self) -> "Fibonacci":
        return self
    def __next__(self) -> int:
        if self._remaining <= 0:
            raise StopIteration
        self._remaining -= 1
        value = self._a
        self._a, self._b = self._b, self._a + self._b
        return value


assert list(Fibonacci(7)) == [0, 1, 1, 2, 3, 5, 8]
print("✅ 19.2 OK")

# --- Exercise 19.3: Generator Expressions ⭐⭐ ---
def sum_of_even_squares(nums: list[int]) -> int:
    """Compute sum of squares of even numbers using a generator expression."""
    # TODO: Use sum(<generator expression>)
    return sum(n * n for n in nums if n % 2 == 0)


assert sum_of_even_squares([1, 2, 3, 4]) == 20
print("✅ 19.3 OK")

---
## 🧩 Phase 4: JSON Handling
JSON is the most common format for APIs and configuration files.

### Exercises in this section
- **21.1 JSON Basics (`dumps` / `loads`)** ⭐
- **21.2 Config Manager (Save/Load JSON)** ⭐⭐
- **21.3 Custom Serialization (Dataclass → JSON)** ⭐⭐⭐
---

In [ ]:
# ======================================
# Exercises 21.1 to 21.3 (Starter)
# ======================================

from __future__ import annotations

import json
from dataclasses import asdict, dataclass
from pathlib import Path
from typing import Any

# --- Exercise 21.1: JSON Basics ⭐ ---
data = {"name": "Asha", "skills": ["python", "pandas"], "active": True}
text = json.dumps(data, indent=2)
parsed = json.loads(text)
assert parsed["name"] == "Asha"
print("✅ 21.1 OK")

# --- Exercise 21.2: Config Manager (Save/Load JSON) ⭐⭐ ---
def save_json(path: str | Path, cfg: dict[str, Any]) -> None:
    # Single-file mode: avoid filesystem writes; return the JSON string instead.
    return json.dumps(cfg, indent=2)


def load_json(path: str | Path) -> dict[str, Any]:
    return json.loads(Path(path).read_text(encoding="utf-8"))


tmp_path = Path("_tmp_config.json")
save_json(tmp_path, {"theme": "dark", "font": 14})
loaded = load_json(tmp_path)
assert loaded["font"] == 14
tmp_path.unlink(missing_ok=True)
print("✅ 21.2 OK")

# --- Exercise 21.3: Custom Serialization ⭐⭐⭐ ---
@dataclass(frozen=True)
class Person:
    name: str
    age: int


p = Person("Ravi", 22)
p_json = json.dumps(asdict(p))
assert json.loads(p_json) == {"name": "Ravi", "age": 22}
print("✅ 21.3 OK")

---
## 🧱 Phase 6 (Beginner): Data Structures (Queue & Binary Tree)
These are classic building blocks used in real software (task scheduling, parsers, search, etc.).

### Exercises in this section
- **28b Queue (FIFO)** ⭐⭐
- **28e Binary Trees (Traversal)** ⭐⭐⭐
---

In [ ]:
# ======================================
# Exercises 28b and 28e (Starter)
# ======================================

from __future__ import annotations

from dataclasses import dataclass
from typing import Any

# --- Exercise 28b: Queue (FIFO) ⭐⭐ ---
class Queue:
    """A simple FIFO queue.

    Engineering note: list.pop(0) is O(n). For real apps use collections.deque.
    """
    def __init__(self):
        self._items: list[Any] = []
    def enqueue(self, item: Any) -> None:
        self._items.append(item)
    def dequeue(self) -> Any:
        if not self._items:
            raise IndexError("dequeue from empty queue")
        return self._items.pop(0)
    def is_empty(self) -> bool:
        return len(self._items) == 0
    def __len__(self) -> int:
        return len(self._items)


q = Queue()
q.enqueue("task1")
q.enqueue("task2")
assert q.dequeue() == "task1"
assert len(q) == 1
print("✅ 28b OK")

# --- Exercise 28e: Binary Trees (Traversal) ⭐⭐⭐ ---
@dataclass
class TreeNode:
    value: int
    left: "TreeNode | None" = None
    right: "TreeNode | None" = None


def inorder(root: TreeNode | None) -> list[int]:
    """Return inorder traversal: left, root, right."""
    if root is None:
        return []
    return inorder(root.left) + [root.value] + inorder(root.right)


tree = TreeNode(2, left=TreeNode(1), right=TreeNode(3))
assert inorder(tree) == [1, 2, 3]
print("✅ 28e OK")

---
## 🧠 Phase 6 (Beginner): Algorithms (Sort & Search)
Sorting and searching are core algorithmic skills.

### Exercises in this section
- **29b Quick Sort** ⭐⭐⭐
- **30a Linear Search** ⭐
- **30b Binary Search** ⭐⭐
---

In [ ]:
# ======================================
# Exercises 29b to 30b (Starter)
# ======================================

from __future__ import annotations

# --- Exercise 29b: Quick Sort ⭐⭐⭐ ---
def quick_sort(nums: list[int]) -> list[int]:
    """Return a new sorted list using quick sort."""
    # TODO: Implement quick sort (recursive)
    if len(nums) <= 1:
        return list(nums)
    pivot = nums[len(nums) // 2]
    left = [x for x in nums if x < pivot]
    middle = [x for x in nums if x == pivot]
    right = [x for x in nums if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)


assert quick_sort([3, 1, 2, 1]) == [1, 1, 2, 3]
print("✅ 29b OK")

# --- Exercise 30a: Linear Search ⭐ ---
def linear_search(nums: list[int], target: int) -> int:
    """Return index of target or -1."""
    for i, value in enumerate(nums):
        if value == target:
            return i
    return -1


assert linear_search([5, 8, 1], 8) == 1
assert linear_search([5, 8, 1], 9) == -1
print("✅ 30a OK")

# --- Exercise 30b: Binary Search ⭐⭐ ---
def binary_search(sorted_nums: list[int], target: int) -> int:
    """Binary search in a sorted list. Return index or -1."""
    lo = 0
    hi = len(sorted_nums) - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        if sorted_nums[mid] == target:
            return mid
        if sorted_nums[mid] < target:
            lo = mid + 1
        else:
            hi = mid - 1
    return -1


assert binary_search([1, 3, 5, 7], 5) == 2
assert binary_search([1, 3, 5, 7], 2) == -1
print("✅ 30b OK")

---
## 📁 Phase 7: File I/O
Reading/writing files is essential for scripts, automation, and data pipelines.

### Exercises in this section
- **31.1 Read & Write a File** ⭐
- **31.2 Multiple Files Pipeline** ⭐⭐
---

In [ ]:
# ======================================
# Exercises 31.1 to 31.2 (Starter)
# ======================================

from __future__ import annotations

from pathlib import Path

# --- Exercise 31.1: Read/Write Files ⭐ ---
path = Path("_tmp_hello.txt")
path.write_text("Hello file!", encoding="utf-8")
text = path.read_text(encoding="utf-8")
assert text == "Hello file!"
path.unlink(missing_ok=True)
print("✅ 31.1 OK")

# --- Exercise 31.2: Multiple Files Pipeline ⭐⭐ ---
p1 = Path("_tmp_a.txt")
p2 = Path("_tmp_b.txt")
p1.write_text("A", encoding="utf-8")
p2.write_text("B", encoding="utf-8")
combined = p1.read_text(encoding="utf-8") + "\n" + p2.read_text(encoding="utf-8")
assert combined == "A\nB"
p1.unlink(missing_ok=True)
p2.unlink(missing_ok=True)
print("✅ 31.2 OK")

---
## 🕒 Phase 7 (Continued): Date & Time
You’ll learn to work with dates, do date arithmetic, and handle time zones.

### Exercises in this section
- **32.1 Current Date/Time** ⭐
- **32.2 Date Arithmetic (Days Between)** ⭐⭐
- **32.3 Timezones (UTC ↔ IST)** ⭐⭐⭐
---

In [ ]:
# ======================================
# Exercises 32.1 to 32.3 (Starter)
# ======================================

from __future__ import annotations

import datetime

# --- Exercise 32.1: Current Date/Time ⭐ ---
now = datetime.datetime.now()
print("Now:", now.isoformat(timespec="seconds"))
print("✅ 32.1 OK")

# --- Exercise 32.2: Date Arithmetic ⭐⭐ ---
d1 = datetime.date(2000, 1, 1)
d2 = datetime.date(2000, 1, 31)
days = (d2 - d1).days
assert days == 30
print("Days between:", days)
print("✅ 32.2 OK")

# --- Exercise 32.3: Timezones (UTC ↔ IST) ⭐⭐⭐ ---
utc = datetime.datetime(2026, 1, 21, 12, 0, 0, tzinfo=datetime.timezone.utc)
ist = datetime.timezone(datetime.timedelta(hours=5, minutes=30))
utc_to_ist = utc.astimezone(ist)
assert utc_to_ist.hour == 17 and utc_to_ist.minute == 30
print("UTC:", utc)
print("IST:", utc_to_ist)
print("✅ 32.3 OK")

---
## 🧮 Phase 7 (Continued): Math Module & Numerical Methods
The `math` module powers engineering/science calculations. Numerical methods help you solve equations when there’s no simple formula.

### Exercises in this section
- **33.1 Math Functions** ⭐
- **33.2 Engineering Math (Projectile Range)** ⭐⭐
- **33.3 Numerical Methods (Newton–Raphson)** ⭐⭐⭐
---

In [ ]:
# ======================================
# Exercises 33.1 to 33.3 (Starter)
# ======================================

from __future__ import annotations

import math
from collections.abc import Callable

# --- Exercise 33.1: Math Functions ⭐ ---
x = 9
print("sqrt:", math.sqrt(x))
print("sin:", round(math.sin(x), 4))
print("cos:", round(math.cos(x), 4))
print("factorial(5):", math.factorial(5))
print("✅ 33.1 OK")

# --- Exercise 33.2: Engineering Math (Projectile Range) ⭐⭐ ---
def projectile_range(speed: float, angle_degrees: float, g: float = 9.81) -> float:
    """Range: R = v^2 * sin(2θ) / g"""
    theta = math.radians(angle_degrees)
    return (speed**2) * math.sin(2 * theta) / g


r = projectile_range(10, 45)
assert r > 0
print("Range for v=10 m/s, 45°:", round(r, 2), "m")
print("✅ 33.2 OK")

# --- Exercise 33.3: Newton–Raphson ⭐⭐⭐ ---
def newton_raphson(
    f: Callable[[float], float],
    df: Callable[[float], float],
    x0: float,
    *,
    max_iter: int = 50,
    tol: float = 1e-10,
 ) -> float:
    x = x0
    for _ in range(max_iter):
        y = f(x)
        if abs(y) <= tol:
            return x
        dy = df(x)
        if dy == 0:
            raise ZeroDivisionError("derivative was zero")
        x = x - (y / dy)
    return x


root2 = newton_raphson(lambda t: t * t - 2, lambda t: 2 * t, x0=1.0)
assert abs(root2 - math.sqrt(2)) < 1e-8
print("sqrt(2) approx:", root2)
print("✅ 33.3 OK")

---
# 🎯 ADVANCED SECTION: Real-World Python Applications
*The following exercises are ADVANCED (⭐⭐⭐) - tackle these after mastering beginner exercises!*

---

## Module 1: Variables & Data Types (Advanced)
**Concept**: Python is dynamically typed - variables can hold any type. In real-world applications, you often receive messy data that needs careful handling.

> **🚀 Advanced Challenge: Data Sanitization**
> 
> **Scenario**: You're building a weather monitoring system. Sensors send data, but sometimes they malfunction and send corrupt values.
> 
> **Task**: Create a data cleaning function that:
> 1. Converts valid numbers to `float` (like "25.5" → 25.5)
> 2. Replaces `None` or empty strings with `0.0`
> 3. Discards corrupted data (like "error", "NaN") and counts errors
> 4. **Constraint**: Use Python's "try-except" pattern for error handling
> 
> **Real-world use**: IoT sensors, data pipelines, API responses
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires understanding of exception handling)

In [ ]:
def clean_sensor_data_v2(raw_data: list) -> tuple[list[float], int]:
    """
    Sanitizes raw sensor data streams by converting to float and handling errors.
    
    Args:
        raw_data (list): A list of mixed-type inputs (int, float, str, None).
        
    Returns:
        tuple[list[float], int]: A tuple containing the list of cleaned floats
                                 and the count of discarded invalid entries.
    """
    valid_data = []
    discarded = 0
    
    for x in raw_data:
        # 1. Handle Missing/Empty
        if x is None or (isinstance(x, str) and x.strip() == ""):
            valid_data.append(0.0)
            continue
        
        try:
            # 2. Attempt Conversion
            valid_data.append(float(x))
        except (ValueError, TypeError):
            # 3. Log Error
            discarded += 1
            
    return valid_data, discarded

# Test Data
raw_stream = [23.5, " 45.1 ", 100, None, "ERR_404", "", 0, "12.5e2"]

cleaned, drops = clean_sensor_data_v2(raw_stream)
print(f"Input: {raw_stream}")
print(f"Cleaned: {cleaned}")
print(f"Discarded count: {drops}")

## Module 2: Strings (Advanced Text Processing)
**Concept**: Strings are immutable sequences of characters. Chaining string methods is a powerful technique for text processing.

> **🚀 Advanced Challenge: Server Log Parser**
>
> **Scenario**: You're building a log analyzer for a web application. Logs are formatted as:
> `"ERROR|2026-01-15|Memory Leak|Server A"`
> 
> **Task**: Parse logs to extract:
> 1. Timestamp (date)
> 2. Severity level (ERROR, WARNING, INFO)
# ✅ SOLUTION: Exercise 7.1

> 3. Error message
> 4. Return structured data as a `dict`
> 
> **Real-world use**: DevOps monitoring, debugging, analytics
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires string splitting and parsing)

In [ ]:
import re

def scrape_contacts(text: str) -> dict:
    """
    Extracts emails and phone numbers using regex patterns.
    """
    # Email Pattern: Simple alphanumeric + @ + domain
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    
    # Phone Pattern: Matches (123) 456-7890, 123-456-7890, +91...
    phone_pattern = r'\(?\+?\d{1,4}\)?[\s.-]?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}'
    
    emails = re.findall(email_pattern, text)
    phones = re.findall(phone_pattern, text)
    
    return {
        "emails": list(set(emails)), # Unique
        "phones": list(set(phones))
    }

dump_text = """
Contact us at support@tech-corp.com or sales@tech.io.
Call output hotlines: (555) 123-4567 or 555-987-6543.
Manager: +91 9998887777.
Ignore invalid@email or 12345.
"""

print(scrape_contacts(dump_text))

## Module 3: Regular Expressions (Advanced Text Processing)
**Concept**: Regular Expressions (regex) are powerful tools for finding patterns in text - like finding all emails in a document.

> **🚀 Advanced Challenge: Contact Information Extractor**
>
> **Scenario**: You're building a business card scanner app that extracts contact info from messy text.
> 
> **Task**: Use Python's `re` module to extract:
> 1. Email addresses (like john@example.com)
> 2. Phone numbers in multiple formats:
>    - (123) 456-7890
# ✅ SOLUTION: Exercise 7.2

>    - 123-456-7890
>    - +91 9876543210
> 3. Return a clean dictionary with all found contacts
> 
> **Real-world use**: Web scraping, data extraction, form validation
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires regex pattern knowledge)

In [ ]:
def parse_logs(logs: list[str]) -> list[dict]:
    """
    Parses legacy log formats without using regex for performance.
    
    Args:
        logs (list[str]): List of raw log strings.
        
    Returns:
        list[dict]: Structured record of logs.
    """
    parsed_results = []
    
    for line in logs:
        # Validation: Must start with [
        if not line.strip().startswith("["):
            continue
            
        try:
            # 1. Extract Timestamp
            close_bracket = line.find("]")
            if close_bracket == -1: continue
            
            timestamp = line[1:close_bracket]
            
            # 2. Extract Level
            # Post-timestamp part: " INFO - Message..."
            remainder = line[close_bracket+1:].strip()
            if " - " in remainder:
                level_part, message_part = remainder.split(" - ", 1)
                level = level_part.upper().strip()
            else:
                continue # Malformed
            
            # 3. Extract ID
            # Check if "ID:" exists in the message
            record_id = None
            if "ID:" in message_part:
                # Get text after ID:
                potential_id = message_part.split("ID:")[-1].strip()
                # Validation: ID must be pure digits
                if potential_id.isdigit():
                    record_id = int(potential_id)
            
            parsed_results.append({
                "time": timestamp,
                "level": level,
                "id": record_id
            })
            
        except ValueError:
            continue
            
    return parsed_results

# Execution
log_lines = [
    "[2023-10-01 12:00] INFO - System started ID:4421",
    "[2023-10-01 12:05] error - Connection failed ID:invalid",
    "[2023-10-01 12:06] Warn - Retrying connection",  # No ID
    "Invalid Line Format"
]

results = parse_logs(log_lines)
for r in results:
    print(r)

## Module 4: Lists (Advanced Data Compression)
**Concept**: Lists are mutable containers that can store any type. Advanced use cases include data compression.

> **🚀 Advanced Challenge: Sparse Matrix Storage**
>
> **Scenario**: You're building a game engine. Game maps are mostly empty space (zeros), wasting memory.
> 
> **Task**: Compress a 2D grid using COO (Coordinate) format:
> - Store only non-zero values as `[(row, col, value)]`
> - Use list comprehension for efficiency
> - Save memory by ignoring zeros
> 
> **Real-world use**: Game development, scientific computing, AI
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires nested loops and list comprehensions)

In [ ]:
dense_matrix = [
    [0, 0, 3, 0],
    [5, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 2, 0, 0]
]

# Pythonic Implementation (List Comprehension)
# Iterates rows (i), then cols (j), identifying non-zero values
sparse_coo = [
    (i, j, row[j]) 
    for i, row in enumerate(dense_matrix) 
    for j in range(len(row)) 
    if row[j] != 0
]

print("Original Matrix:")
for row in dense_matrix: print(row)

print("\nCompressed (Row, Col, Value) Tuples:")
print(sparse_coo)

# Verification: Reconstruct Matrix
reconstructed = [[0]*4 for _ in range(4)]
for r, c, v in sparse_coo:
    reconstructed[r][c] = v
    
print("\nReconstructed match:", dense_matrix == reconstructed)

## Module 5: Dictionaries (Advanced Search Systems)
**Concept**: Dictionaries are key-value pairs with super-fast lookups (O(1) time complexity).

> **🚀 Advanced Challenge: Build a Search Engine Index**
>
> **Scenario**: You're building a mini search engine like Google. You need to quickly find which documents contain specific words.
> 
> **Task**: Create an "inverted index" - a dictionary mapping:
> - Each word → set of document IDs containing that word
> - Filter out common words ("is", "the", "for")
> - Use `setdefault()` for clean code
> 
> **Real-world use**: Search engines, document databases, autocomplete
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires dictionary operations and sets)

In [ ]:
def build_inverted_index(docs: dict[int, str], stops: set[str]) -> dict[str, set[int]]:
    """
    Constructs an inverted index from a corpus of documents.
    
    Args:
        docs (dict): Document ID mapped to text content.
        stops (set): Words to ignore.
    """
    index = {} # dict of word -> set of IDs
    
    for doc_id, text in docs.items():
        # Clean and split
        words = text.lower().split()
        
        for word in words:
            if word in stops:
                continue
            
            # Logic: If word not in dict, create value as Set. Then add ID.
            # setdefault returns the value (the set), allowing .add() chaining
            index.setdefault(word, set()).add(doc_id)
            
    return index

documents = {
    101: "Python is great for data analysis",
    102: "Data analysis requires statistics",
    103: "Python code is readable code"
}
stop_words = {"is", "for", "the"}

inverted = build_inverted_index(documents, stop_words)

print("Inverted Index (Word -> {Doc IDs}):")
for word, doc_ids in inverted.items():
    print(f"{word.ljust(10)} : {doc_ids}")

## Module 6: Control Flow (Advanced Error Handling)
**Concept**: Control flow (`if`, `for`, `while`) lets programs make intelligent decisions.

> **🚀 Advanced Challenge: Smart Retry System**
>
> **Scenario**: You're connecting to an unreliable API (like a payment gateway). It fails randomly.
> 
> **Task**: Build a retry system with "exponential backoff":
> 1. Try to connect
> 2. If it fails, wait 2^attempt seconds (1s, 2s, 4s, 8s...)
> 3. Give up after MAX_RETRIES attempts
> 4. Return success/failure status
> 
> **Real-world use**: API clients, network programming, microservices
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires loops, conditionals, and timing)

In [ ]:
import random
import time

def unstable_network_request() -> bool:
    """Simulates a connection that fails 70% of the time."""
    return random.random() > 0.7

def retry_handler(max_retries: int = 5):
    """
    Attempts to connect with exponential backoff strategy.
    """
    attempt = 0
    success = False
    
    print(f"Starting connection attempts (Max: {max_retries})...")
    
    while attempt < max_retries:
        print(f"Attempt {attempt + 1}...", end=" ")
        
        if unstable_network_request():
            print("SUCCESS! Connected.")
            success = True
            break
        else:
            # Exponential Backoff Calculation
            wait_time = 2 ** attempt 
            print(f"Failed. Backing off for {wait_time}s.")
            attempt += 1
            
    if not success:
        print("CRITICAL: Connection timed out after all retries.")

retry_handler(max_retries=4)

## Module 7: Functional Programming (Advanced Data Pipelines)
**Concept**: Functions are "first-class" in Python - they can be passed as arguments like any other variable.

> **🚀 Advanced Challenge: Data Processing Pipeline**
>
> **Scenario**: You're building a data cleaning system like in Excel or SQL. Data needs multiple transformations.
> 
> **Task**: Create a pipeline function that:
> 1. Takes raw data and multiple transformation functions
> 2. Applies each function in sequence (like a chain)
> 3. Example pipeline: Filter → Normalize → Round
> 
> **Example**:
> ```python
> data = [10, 20, 50, 40, 30]
> pipeline(data, filter_func, normalize_func, round_func)
> # → [0.0, 0.5, 1.0, 0.75]
> ```
> 
> **Real-world use**: ETL systems, data analysis, machine learning preprocessing
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires understanding functions as arguments)

In [ ]:
def process_pipeline(data: list, *steps):
    """
    Passes data through a series of transformation functions sequentially.
    """
    result = data
    for step_func in steps:
        result = step_func(result)
    return result

def normalize_scale(x_list: list[float]) -> list[float]:
    """Min-Max scaler: (x - min) / (max - min)"""
    m_min = min(x_list)
    m_max = max(x_list)
    return [(i - m_min) / (m_max - m_min) for i in x_list]

data_source = [10, 20, 50, 40, 30]

# Pipeline Construction
final_output = process_pipeline(
    data_source,
    lambda x: [i for i in x if i >= 20],      # Step 1: Filter
    normalize_scale,                          # Step 2: Scale
    lambda x: [round(i, 2) for i in x]        # Step 3: Format
)

print(f"Original: {data_source}")
print(f"Processed: {final_output}")

## Module 8: Object-Oriented Programming (Advanced Class Design)
**Concept**: Classes are blueprints for creating objects. They bundle data and functions together.

> **🚀 Advanced Challenge: Build a Vector Math Library**
>
> **Scenario**: You're building a 3D game engine or physics simulator. You need a `Vector` class for calculations.
> 
> **Task**: Create a `Vector` class that:
> 1. Stores 3D coordinates (x, y, z)
> 2. Supports addition: `v1 + v2`
> 3. Calculates magnitude (length): `v1.magnitude()`
> 4. Bonus: Create a `UnitVector` subclass (length = 1)
> 
> **Example**:
> ```python
> v1 = Vector(3, 4, 0)
> v2 = Vector(1, 1, 1)
> v3 = v1 + v2  # → Vector(4, 5, 1)
> print(v1.magnitude())  # → 5.0
> ```
> 
> **Real-world use**: Game development, robotics, computer graphics, physics
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires OOP concepts like inheritance and operator overloading)

In [ ]:
import math

class Vector:
    """Represents a 3D Vector."""
    
    def __init__(self, x, y, z):
        self.components = [x, y, z]
        
    @property
    def x(self): return self.components[0]
    @property
    def y(self): return self.components[1]
    @property
    def z(self): return self.components[2]
        
    def __add__(self, other):
        """Supports Vector+Vector and Vector+Scalar addition."""
        if isinstance(other, Vector):
            return Vector(self.x + other.x, self.y + other.y, self.z + other.z)
        elif isinstance(other, (int, float)):
            return Vector(self.x + other, self.y + other, self.z + other)
        else:
            raise TypeError("Operand must be Vector or Scalar")
    
    def magnitude(self) -> float:
        return math.sqrt(sum(c**2 for c in self.components))
    
    def __str__(self):
        return f"Vec({self.x:.2f}, {self.y:.2f}, {self.z:.2f})"

class UnitVector(Vector):
    """A Vector that maintains a magnitude of 1.0."""
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.normalize()
        
    def normalize(self):
        mag = self.magnitude()
        if mag == 0: raise ValueError("Zero vector cannot be normalized")
        self.components = [c / mag for c in self.components]

# Test
v1 = Vector(3, 4, 0)
v2 = Vector(1, 1, 1)

print(f"Add Vector: {v1 + v2}")
print(f"Add Scalar: {v1 + 10}")

u1 = UnitVector(3, 4, 0)
print(f"Unit Vector: {u1} | Mag: {u1.magnitude()}")

## Module 9: File Operations (Advanced Safe Writing)
**Concept**: The `with` statement ensures files are properly closed. Atomic operations prevent file corruption.

> **🚀 Advanced Challenge: Crash-Safe File Writing**
>
> **Scenario**: You're building a config editor. If the program crashes mid-write, the config file gets corrupted and the app won't start!
> 
> **Task**: Implement `atomic_write(filename, data)` that:
> 1. Writes to a temporary `.tmp` file first
> 2. Only replaces the real file if write succeeds
> 3. Cleans up temp files on failure
> 4. Guarantees: File is never left in broken state
> 
> **Example**:
> ```python
> atomic_write("config.json", data)
> # Even if crash happens, config.json is intact!
> ```
> 
> **Real-world use**: Database transactions, config managers, save systems
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires exception handling and file system operations)

In [ ]:
ENABLE_FILESYSTEM_DEMOS = False  # Set True only in a safe sandbox

import os
import random

def atomic_write(filename: str, content: str):
    """
    Writes to a file atomically to prevent corruption during crashes.
    """
    temp_filename = filename + ".tmp"
    
    try:
        # 1. Write to temp file
        print(f"Writing to temp file: {temp_filename}")
        with open(temp_filename, "w") as f:
            f.write(content)
            
            # Simulate crash risk
            if random.random() < 0.1: 
                raise RuntimeError("Simulated Crash during write!")
        
        # 2. Swap (Atomic)
        os.replace(temp_filename, filename)
        print(f"Success: Committed to {filename}")
        
    except Exception as e:
        print(f"Error: {e}")
        # Cleanup
        if os.path.exists(temp_filename):
            os.remove(temp_filename)
            print("Cleanup: Removed corrupted temp file.")


if ENABLE_FILESYSTEM_DEMOS:
    atomic_write("config_v1.txt", "Critical Data")
else:
    print('ℹ️ Filesystem demo disabled (ENABLE_FILESYSTEM_DEMOS=False).')


---
# 📊 DATA SCIENCE SECTION: NumPy, Pandas, Visualization & ML
*These modules require understanding of math and statistics. Complete all beginner exercises first!*

---

## Module 10: NumPy (Advanced Array Operations)
**Concept**: NumPy performs math operations 100x faster than regular Python by using C-level code.

> **🚀 Advanced Challenge: Signal Processing & Anomaly Detection**
>
> **Scenario**: You're analyzing heart rate data from a fitness tracker. You need to smooth noise and find abnormal spikes.
> 
> **Task**: Using NumPy arrays (NO LOOPS!):
> 1. Generate a sine wave (simulates heartbeat) + random noise
> 2. Apply "Moving Average" smoothing using `np.convolve`
> 3. Find anomalies (values > 2 standard deviations from mean)
> 4. Return: smoothed signal + anomaly indices
> 
> **Why NumPy**: Processing 1000 data points takes:
> - Python loops: ~100ms
> - NumPy vectorization: ~1ms (100x faster!)
> 
> **Real-world use**: Medical devices, stock trading, sensor analysis
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires NumPy arrays, statistics knowledge)

In [ ]:
import numpy as np

def signal_processing_lab():
    """Performs signals analysis using vectorized NumPy operations."""
    
    # 1. Setup Data
    np.random.seed(42)
    t = np.linspace(0, 10, 1000)
    pure_signal = np.sin(t)
    noise = np.random.normal(0, 0.5, 1000)
    noisy_signal = pure_signal + noise
    
    # 2. Moving Average Filter (Vectorized)
    window_size = 50
    kernel = np.ones(window_size) / window_size
    filtered_signal = np.convolve(noisy_signal, kernel, mode='same')
    
    # 3. Anomaly Detection
    residuals = np.abs(noisy_signal - filtered_signal)
    threshold = 2 * np.std(residuals)
    
    # Boolean Masking
    anomaly_mask = residuals > threshold
    anomaly_indices = np.where(anomaly_mask)[0]
    
    return {
        "anomalies": len(anomaly_indices),
        "threshold": threshold,
        "indices_sample": anomaly_indices[:5]
    }

print(signal_processing_lab())

## Module 11: Pandas (Advanced Data Analysis)
**Concept**: Pandas is like Excel in Python - but programmable and can handle millions of rows.

> **🚀 Advanced Challenge: IoT Sensor Data Cleaning**
>
> **Scenario**: You're analyzing smart home sensor data (temperature, humidity, pressure). Data has missing values and outliers.
> 
> **Task**: Using Pandas DataFrame:
> 1. Load sensor readings (date, temp, humidity, pressure)
> 2. Handle missing data: fill with mean or drop
> 3. Remove outliers (values > 3 standard deviations)
> 4. Resample data: Group by hour and calculate averages
> 5. Export cleaned data
> 
> **Pandas Operations**:
> ```python
> df.fillna(df.mean())  # Fill missing values
> df.groupby('hour').mean()  # Aggregate
> df[df['temp'] < threshold]  # Filter
> ```
> 
> **Real-world use**: Business analytics, scientific research, data engineering
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires understanding DataFrames, aggregation)

In [ ]:
import pandas as pd
import numpy as np

def analyze_iot_data():
    """Generates and processes simulated IoT sensor data."""
    
    # 1. Create Time Series
    rng = pd.date_range('2023-01-01', periods=60, freq='T')
    
    df = pd.DataFrame({
        'Timestamp': rng,
        'Sensor_A': np.random.uniform(20, 30, 60), 
        'Sensor_B': np.random.randint(100, 200, 60)
    })
    
    # Modern Pattern: Avoid inplace=True
    df = df.set_index('Timestamp')
    
    # Simulate Missing Data
    mask = np.random.choice([True, False], size=len(df), p=[0.2, 0.8])
    df_missing = df.mask(mask)
    
    # 2. Repair & Resample
    df_clean = df_missing.interpolate(method='time')
    
    df_resampled = df_clean.resample('10T').agg({
        'Sensor_A': 'mean',
        'Sensor_B': 'max'
    })
    
    return df_clean, df_resampled

clean_df, agg_df = analyze_iot_data()
print(agg_df)

## Module 12: Matplotlib (Advanced Data Visualization)
**Concept**: Matplotlib creates professional charts and graphs. The `fig, ax` pattern gives precise control.

> **🚀 Advanced Challenge: Multi-Panel Dashboard**
>
> **Scenario**: You're building a data analytics dashboard (like Excel charts or Tableau).
> 
> **Task**: Create a 2x2 grid of subplots showing:
> 1. **Top-Left**: Line chart (temperature over time)
> 2. **Top-Right**: Histogram (distribution of pressure readings)
> 3. **Bottom-Left**: Scatter plot (correlation: temp vs humidity)
> 4. **Bottom-Right**: Bar chart (average readings by hour)
> 
> **Matplotlib Code Pattern**:
> ```python
> fig, axs = plt.subplots(2, 2, figsize=(12, 8))
> axs[0, 0].plot(x, y)  # Top-left
> axs[0, 1].hist(data)  # Top-right
> ```
> 
> **Real-world use**: Business reporting, scientific papers, data journalism
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires subplot management, chart customization)

In [ ]:
import matplotlib.pyplot as plt

def plot_dashboard(df, df_agg):
    """Generates a 4-panel engineering dashboard."""
    fig, axs = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle('IoT Sensor Analysis Dashboard', fontsize=16)

    # 1. Line Chart
    axs[0, 0].plot(df.index, df['Sensor_A'], color='tab:blue')
    axs[0, 0].set_title('Sensor A: Time Series')

    # 2. Histogram
    axs[0, 1].hist(df['Sensor_B'].dropna(), bins=15, color='tab:orange', alpha=0.7)
    axs[0, 1].set_title('Sensor B Distribution')

    # 3. Scatter
    axs[1, 0].scatter(df['Sensor_A'], df['Sensor_B'], alpha=0.5, c='teal')
    axs[1, 0].set_title('Correlation: Temp vs Pressure')

    # 4. Bar Chart
    labels = [t.strftime('%H:%M') for t in df_agg.index]
    axs[1, 1].bar(labels, df_agg['Sensor_A'], color='tab:purple')
    axs[1, 1].set_title('10-min Avg Temp')

    plt.tight_layout()
    plt.show()

# Run
plot_dashboard(clean_df, agg_df)

## Module 13: Machine Learning (Classification)
**Concept**: Machine Learning trains computers to make predictions from data patterns.

> **🚀 Advanced Challenge: Spam Email Classifier**
>
> **Scenario**: You're building an email filter like Gmail's spam detector.
> 
> **Task**: Train a Decision Tree classifier to detect spam:
> 1. Load labeled email data (spam vs not-spam)
> 2. Extract features (word counts, length, special chars)
> 3. Split data: 80% training, 20% testing
> 4. Train the classifier and predict on test data
> 5. Evaluate accuracy with confusion matrix
> 
> **ML Workflow**:
> ```python
> # 1. Prepare data
> X_train, X_test, y_train, y_test = train_test_split(X, y)
> 
> # 2. Train model
> model = DecisionTreeClassifier()
> model.fit(X_train, y_train)
> 
> # 3. Predict & evaluate
> predictions = model.predict(X_test)
> accuracy = accuracy_score(y_test, predictions)
> ```
> 
> **Real-world use**: Email filters, fraud detection, medical diagnosis, recommendation systems
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires ML concepts, scikit-learn library)

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

def advanced_ml_evaluation():
    # 1. Data
    X, y = make_moons(n_samples=500, noise=0.3, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # 2. Model
    dt = DecisionTreeClassifier(max_depth=5)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)
    
    # 3. Evaluation
    print("Classification Report:\n", classification_report(y_test, y_pred))
    
    # Confusion Matrix Heatmap
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title("Confusion Matrix")
    plt.show()
    
    # 4. Cross Validation (Robustness check)
    scores = cross_val_score(dt, X, y, cv=5)
    print(f"5-Fold CV Accuracy: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

advanced_ml_evaluation()

---
# 🗄️ DATABASE & ALGORITHMS SECTION
*These modules cover databases and computer science fundamentals*

---

## Module 14: SQL Databases (Relational Data)
**Concept**: SQL (Structured Query Language) manages data in tables with relationships between them.

> **🚀 Advanced Challenge: Library Management System**
>
> **Scenario**: You're building a library database that tracks authors and their books.
> 
> **Task**: Using SQLite (built into Python):
> 1. Create two tables: `Authors` and `Books`
> 2. Establish relationship: Each book has one author
> 3. Write a query to find:
>    - Authors with more than 1 book
>    - Their total book count
>    - Year of their latest book
> 
> **SQL Concepts**:
> ```sql
> -- Join two tables
> SELECT a.name, COUNT(b.id) 
> FROM Authors a 
> JOIN Books b ON a.id = b.author_id
> GROUP BY a.id
> ```
> 
> **Real-world use**: Web apps, inventory systems, banking, social media
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires SQL joins, aggregation)

In [ ]:
import sqlite3
import pandas as pd

def library_sql_lab():
    """Demonstrates complex SQL queries using in-memory SQLite."""
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()
    
    # Schema & Data
    cursor.execute('CREATE TABLE Authors (id INTEGER PRIMARY KEY, name TEXT)')
    cursor.execute('CREATE TABLE Books (id INTEGER PRIMARY KEY, title TEXT, year INT, author_id INT)')
    
    cursor.executemany("INSERT INTO Authors VALUES (?,?)", [(1, "Asimov"), (2, "Tolkien")])
    cursor.executemany("INSERT INTO Books VALUES (?,?,?,?)", [
        (1, "Foundation", 1951, 1), (2, "I, Robot", 1950, 1),
        (3, "Hobbit", 1937, 2), (4, "LOTR", 1954, 2)
    ])
    
    # Aggregation Query
    query = """
    SELECT a.name, COUNT(b.id) as count, MAX(b.year) as latest
    FROM Authors a JOIN Books b ON a.id = b.author_id
    GROUP BY a.id HAVING count > 1
    """
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

print(library_sql_lab())

## Module 15: NoSQL Databases (Document Storage)
**Concept**: NoSQL databases store data as flexible documents (like JSON) instead of rigid tables.

> **🚀 Advanced Challenge: Build a Mini MongoDB**
>
> **Scenario**: You're building a document database for a blog platform that stores user profiles with varying fields.
> 
> **Task**: Implement a `MockCollection` class that supports MongoDB-like queries:
> 1. Store documents as Python dicts
> 2. Support queries like: `{"age": {"$gt": 25}}`  (find users over 25)
> 3. Support nested field queries: `{"meta.login": "today"}`
> 4. Return matching documents
> 
> **Why NoSQL**: Unlike SQL tables (all rows have same columns), NoSQL allows:
> ```python
> user1 = {"id": 1, "age": 30}
> user2 = {"id": 2, "age": 25, "premium": True}  # Extra field OK!
> ```
> 
> **Real-world use**: MongoDB, Firebase, AWS DynamoDB
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires nested data traversal, operator logic)

In [ ]:
class MockCollection:
    """Simulates a MongoDB Collection with a basic find() parser."""
    
    def __init__(self, data):
        self.data = data
        
    def find(self, query):
        results = []
        for doc in self.data:
            match = True
            for key, condition in query.items():
                # Nested Key resolution (e.g., 'meta.login')
                val = doc
                for k in key.split('.'):
                    val = val.get(k) if isinstance(val, dict) else None
                
                # Operator logic
                if isinstance(condition, dict):
                    if "$gt" in condition and not (val > condition["$gt"]): match = False
                elif val != condition:
                    match = False
            
            if match: results.append(doc)
        return results

db = MockCollection([
    {"id": 1, "age": 30, "meta": {"login": "today"}},
    {"id": 2, "age": 20, "meta": {"login": "yesterday"}}
])

print(db.find({"age": {"$gt": 25}, "meta.login": "today"}))

## Module 16: Stack Data Structure (LIFO)
**Concept**: Stacks are Last-In-First-Out (LIFO) - like a stack of plates. Critical for undo/redo, browser history.

> **🚀 Advanced Challenge: MinStack (Constant-Time Min)**
>
> **Scenario**: You're building a text editor's undo system that also tracks the "minimum edit cost."
> 
> **Task**: Implement a Stack with special requirement:
> 1. Standard operations: `push(x)`, `pop()`, `top()`
> 2. **Special**: `get_min()` returns minimum element in O(1) time
> 3. **Constraint**: Cannot scan entire stack (must be instant!)
> 
> **Trick**: Use auxiliary "min stack" to track minimums
> 
> **Example**:
> ```python
> stack.push(5)  # Stack: [5], Min: 5
> stack.push(2)  # Stack: [5,2], Min: 2
> stack.push(10) # Stack: [5,2,10], Min: 2
> stack.get_min() # Returns 2 in O(1)!
> ```
> 
> **Real-world use**: Expression evaluation, browser back/forward, function call stack
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires understanding auxiliary data structures)

In [ ]:
class MinStack:
    """Stack that allows retrieving the minimum element in O(1)."""
    
    def __init__(self):
        self.stack = []     # Main stack
        self.min_stack = [] # Aux stack
        
    def push(self, val):
        self.stack.append(val)
        # Fix: Always ensure min_stack has a corresponding top, 
        # or conditionally push ONLY if new val <= min.
        # Here we use the conditional approach for space efficiency.
        if not self.min_stack or val <= self.min_stack[-1]:
            self.min_stack.append(val)
            
    def pop(self):
        if not self.stack: return
        val = self.stack.pop()
        # Sync: Only pop from min_stack if the values match
        if val == self.min_stack[-1]:
            self.min_stack.pop()
        return val
        
    def get_min(self):
        return self.min_stack[-1] if self.min_stack else None

# Test
ms = MinStack()
[ms.push(x) for x in [5, 2, 10, 1]]
print(f"Min: {ms.get_min()}") # 1
ms.pop() # Pop 1
print(f"Min after pop: {ms.get_min()}") # 2

## Module 17: Graph Algorithms (BFS - Breadth-First Search)
**Concept**: Graphs represent networks (social media, roads, internet). BFS finds shortest paths.

> **🚀 Advanced Challenge: Social Network "Degrees of Separation"**
>
> **Scenario**: You're building LinkedIn's "how you're connected" feature. Find shortest path between two people.
> 
> **Task**: Implement BFS (Breadth-First Search) to:
> 1. Take a graph (dict of connections)
> 2. Find shortest path from person A to person B
> 3. Return the full path: `['Alice', 'Bob', 'Carol', 'Dave']`
> 
> **BFS Algorithm** (Level-by-Level exploration):
> ```python
> 1. Start at source node
> 2. Visit all immediate neighbors (level 1)
> 3. Then visit their neighbors (level 2)
> 4. Continue until goal found
> ```
> 
> **Why BFS finds shortest**: It explores nodes in order of distance!
> 
> **Real-world use**: GPS navigation, social networks, web crawlers, recommendation systems
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires queue data structure, graph traversal)

In [ ]:
from collections import deque

def bfs_shortest_path(graph, start, goal):
    """Finds shortest path using BFS."""
    queue = deque([(start, [start])])
    visited = set([start])
    
    while queue:
        node, path = queue.popleft()
        
        if node == goal: return path
        
        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))
                
    return None

graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D'],
    'C': ['A', 'D'],
    'D': ['B', 'C', 'E'],
    'E': ['D']
}

print(f"Path A->E: {bfs_shortest_path(graph, 'A', 'E')}")

## Module 18: Linked Lists (Dynamic Data Structures)
**Concept**: Linked Lists are chains of nodes, each pointing to the next. Unlike arrays, they can grow/shrink easily.

> **🚀 Advanced Challenge: Detect Cycle in Linked List**
>
> **Scenario**: You're debugging a corrupted data structure where a node accidentally points back, creating an infinite loop!
> 
> **Task**: Implement Floyd's "Tortoise and Hare" algorithm:
> 1. Create a LinkedList class with nodes
> 2. Detect if the list has a cycle (node points back to earlier node)
> 3. **Constraint**: Must use O(1) space (no hash sets!)
> 
> **Floyd's Algorithm** (Genius trick!):
> ```python
> slow = head  # Moves 1 step
> fast = head  # Moves 2 steps
> 
> while fast:
>     slow = slow.next
>     fast = fast.next.next
>     if slow == fast:  # They meet = cycle exists!
>         return True
> ```
> 
> **Real-world use**: Memory management, OS internals, file systems
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires pointer manipulation, algorithm design)

In [ ]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def has_cycle(head: ListNode) -> bool:
    """Floyd's Tortoise and Hare Algorithm."""
    if not head: return False
    
    slow, fast = head, head
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        if slow == fast:
            return True
    return False

# Test
node1 = ListNode(1)
node2 = ListNode(2)
node3 = ListNode(3)
node1.next = node2
node2.next = node3
node3.next = node1 # Cycle

print(f"Cycle detected: {has_cycle(node1)}")

## Module 19: Sorting Algorithms (Merge Sort)
**Concept**: Sorting arranges data in order. Different algorithms have different trade-offs.

> **🚀 Advanced Challenge: Implement Merge Sort from Scratch**
>
> **Scenario**: You're building a database system. Need to sort millions of records efficiently.
> 
> **Task**: Implement Merge Sort (Divide & Conquer algorithm):
> 1. Split array in half recursively until single elements
> 2. Merge sorted halves back together
> 3. Time complexity: O(n log n) - much faster than bubble sort!
> 
> **Merge Sort Algorithm**:
> ```python
> def merge_sort(arr):
>     if len(arr) <= 1: return arr
>     
>     mid = len(arr) // 2
>     left = merge_sort(arr[:mid])   # Sort left half
>     right = merge_sort(arr[mid:])  # Sort right half
>     
>     return merge(left, right)      # Combine
> ```
> 
> **Why Merge Sort?**
> - **Stable**: Preserves order of equal elements
> - **Predictable**: Always O(n log n), unlike QuickSort
> - **External sorting**: Works with data larger than RAM
> 
> **Real-world use**: Database systems, external file sorting, large dataset processing
> 
> **Difficulty**: ⭐⭐⭐ Advanced (requires recursion, divide-and-conquer thinking)

In [ ]:
def merge_sort(arr):
    """
    Recursive implementation of Merge Sort. 
    Complexity: O(n log n). Stable.
    """
    if len(arr) <= 1:
        return arr
    
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    
    return merge(left, right)

def merge(left, right):
    sorted_arr = []
    i = j = 0
    
    # Merge step
    while i < len(left) and j < len(right):
        if left[i] < right[j]:
            sorted_arr.append(left[i])
            i += 1
        else:
            sorted_arr.append(right[j])
            j += 1
            
    sorted_arr.extend(left[i:])
    sorted_arr.extend(right[j:])
    return sorted_arr

data = [38, 27, 43, 3, 9, 82, 10]
print(f"Sorted: {merge_sort(data)}")

# Section 18: Machine Learning
This section introduces Scikit-Learn.
First, ensure you have it installed:



In [ ]:
%pip install scikit-learn


## Exercise 18.1: Linear Regression (Machine Learning)

**Concept**:
Linear Regression predicts a continuous value (e.g., price, temperature) based on input features by fitting a straight line (or plane).

**Challenge**:
1. Import `LinearRegression` from `sklearn.linear_model`.
2. Given `X` (Feature) and `y` (Target), fit the model.
3. Predict the value for `X = [[6]]`.



In [ ]:
# --- Progress (auto) ---
# Status: Not Started
# XP: 50
# -----------------------

import numpy as np
from sklearn.linear_model import LinearRegression

# Training data
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10]) # y = 2x

# 1. Create model
model = None # Your code here

# 2. Fit model
# Your code here

# 3. Predict for 6
prediction = None # Your code here
print("Prediction for 6:", prediction)


<details>
<summary>✅ Solution</summary>

```python
import numpy as np
from sklearn.linear_model import LinearRegression

X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10])

# 1. Create model
model = LinearRegression()

# 2. Fit model
model.fit(X, y)

# 3. Predict for 6
prediction = model.predict([[6]])
print("Prediction for 6:", prediction) # Should be around 12
```
</details>



## Exercise 18.2: Train/Test Split & KNN

**Concept**:
- **Train/Test Split**: We split data into a training set to teach the model and a testing set to evaluate it.
- **K-Nearest Neighbors (KNN)**: A simple classification algorithm that predicts the class based on the majority class of its 'K' nearest neighbors.

**Challenge**:
1. Load the Iris dataset using `load_iris`.
2. Split it using `train_test_split`.
3. Train a `KNeighborsClassifier` with `n_neighbors=3`.
4. Check the score (accuracy).



In [ ]:
# --- Progress (auto) ---
# Status: Not Started
# XP: 60
# -----------------------

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Load data
data = load_iris()
X, y = data.data, data.target

# 1. Split data (test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Train KNN
knn = None # Your code here
# knn.fit(...)

# 3. Score
score = 0 # knn.score(...)
print("Accuracy:", score)


<details>
<summary>✅ Solution</summary>

```python
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

data = load_iris()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

score = knn.score(X_test, y_test)
print("Accuracy:", score)
```
</details>

